In [ ]:
!pip install pydantic pydantic-ai python-dotenv ipython devtools 
!pip install logfire

# Run This for Notebook

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Mapping IMPLEMENTATION

In [ ]:
# Imports and environment variables
import os
import json
import re
from dotenv import load_dotenv
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, List
from devtools import debug
from dataclasses import dataclass
from pydantic_ai import Agent, ModelRetry, RunContext
import logfire 
import pprint
from pydantic_ai.models.openai import OpenAIModel
load_dotenv()
logfire.configure(console=False)
logfire.instrument_openai()

# Get API keys from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "")

#### MODELS

In [ ]:
# Mapping Models
from pydantic import BaseModel, Field, field_validator
from typing import Optional, Literal
from enum import Enum
import re
from datetime import date

# Enums for validation
class XBRLFiling(str, Enum):
    FULL = "Full"
    PARTIAL = "Partial"

class FinancialStatementType(str, Enum):
    COMPANY = "Company"
    CONSOLIDATED = "Consolidated"

class AccountingStandard(str, Enum):
    SFRS = "SFRS"
    SFRS_SE = "SFRS for SE"
    IFRS = "IFRS"
    OTHER = "Other"

class StatementOfFinancialPositionType(str, Enum):
    CLASSIFIED = "Classified"
    LIQUIDITY = "Liquidity-based"

class RoundingLevel(str, Enum):
    THOUSANDS = "Thousands"
    MILLIONS = "Millions"
    UNITS = "Units"

class XBRLPreparationMethod(str, Enum):
    AUTOMATED = "Automated"
    MANUAL = "Manual"
    HYBRID = "Hybrid"

class AuditOpinion(str, Enum):
    UNQUALIFIED = "Unqualified"
    QUALIFIED = "Qualified"
    ADVERSE = "Adverse"
    DISCLAIMER = "Disclaimer"

# Custom validators
def validate_currency_code(v: str) -> str:
    """Validate ISO 4217 currency code"""
    if not re.match(r'^[A-Z]{3}$', v):
        raise ValueError("Currency code must be 3 uppercase letters (ISO 4217)")
    return v

def validate_date(v: str) -> str:
    """Validate ISO 8601 date format"""
    if not re.match(r'^\d{4}-\d{2}-\d{2}$', v):
        raise ValueError("Date must be in ISO 8601 format (YYYY-MM-DD)")
    return v

def validate_uen(v: str) -> str:
    """Validate UEN format"""
    if not re.match(r'^\d{8}[A-Z]$', v):
        raise ValueError("UEN must be 8 digits followed by 1 uppercase letter")
    return v

# Pydantic models
class FilingInformation(BaseModel):
    """Basic information about the entity and the filing"""
    NameOfCompany: str = Field(..., min_length=1, description="Registered name of the entity in BizFile")
    UniqueEntityNumber: str = Field(..., description="Unique Entity Number assigned by ACRA")
    CurrentPeriodStartDate: str = Field(..., description="Start date of the current reporting period")
    CurrentPeriodEndDate: str = Field(..., description="End date of the current reporting period")
    PriorPeriodStartDate: Optional[str] = Field(None, description="Start date of the prior reporting period for comparatives")
    TypeOfXBRLFiling: XBRLFiling = Field(..., description="Whether the filing contains full or partial XBRL information")
    NatureOfFinancialStatementsCompanyLevelOrConsolidated: FinancialStatementType = Field(..., description="Whether the statements are for the company alone or consolidated group")
    TypeOfAccountingStandardUsedToPrepareFinancialStatements: AccountingStandard = Field(..., description="Accounting standards framework used")
    DateOfAuthorisationForIssueOfFinancialStatements: str = Field(..., description="Date when the financial statements were authorized for issue")
    TypeOfStatementOfFinancialPosition: StatementOfFinancialPositionType = Field(..., description="Whether the statement of financial position is presented in current/non-current format or order of liquidity")
    WhetherTheFinancialStatementsArePreparedOnGoingConcernBasis: bool = Field(..., description="Whether the entity is a going concern")
    WhetherThereAreAnyChangesToComparativeAmounts: Optional[bool] = Field(None, description="Whether comparative amounts have been restated or reclassified")
    DescriptionOfPresentationCurrency: str = Field(..., description="Currency used for presentation of the financial statements")
    DescriptionOfFunctionalCurrency: str = Field(..., description="Primary currency of the economic environment in which the entity operates")
    LevelOfRoundingUsedInFinancialStatements: RoundingLevel = Field(..., description="Level of rounding applied to the financial data")
    DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities: str = Field(
        ..., 
        min_length=20, 
        max_length=100,
        description="Provide a detailed description of the nature of the entity's operations and its principal business activities, including key operational insights."
    )
    
    PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice: str = Field(..., description="Main location where business is conducted")
    WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees: bool = Field(..., description="Whether the company or group has more than 50 employees")
    NameOfParentEntity: Optional[str] = Field(None, description="Immediate parent company name")
    NameOfUltimateParentOfGroup: Optional[str] = Field(None, description="Ultimate parent company name")
    TaxonomyVersion: Literal["2022.2"] = Field(..., description="Version of the XBRL taxonomy used")
    NameAndVersionOfSoftwareUsedToGenerateXBRLFile: str = Field(..., description="Software used to prepare the XBRL filing")
    HowWasXBRLFilePrepared: XBRLPreparationMethod = Field(XBRLPreparationMethod.AUTOMATED, description="Indicate how the XBRL file was prepared: automated, manual, or hybrid (default is automated).")
    
    # Pydantic V2 field validators replacing the deprecated @validator decorators
    @field_validator("UniqueEntityNumber", mode="before")
    def validate_uen_field(cls, v: str) -> str:
        return validate_uen(v)
    
    @field_validator("DescriptionOfPresentationCurrency", mode="before")
    def validate_presentation_currency(cls, v: str) -> str:
        return validate_currency_code(v)
    
    @field_validator("DescriptionOfFunctionalCurrency", mode="before")
    def validate_functional_currency(cls, v: str) -> str:
        return validate_currency_code(v)
    
    @field_validator("CurrentPeriodStartDate", mode="before")
    def validate_current_period_start(cls, v: str) -> str:
        return validate_date(v)
    
    @field_validator("CurrentPeriodEndDate", mode="before")
    def validate_current_period_end(cls, v: str) -> str:
        return validate_date(v)
    
    @field_validator("PriorPeriodStartDate", mode="before")
    def validate_prior_period_start(cls, v: Optional[str]) -> Optional[str]:
        if v is None:
            return v
        return validate_date(v)
    
    @field_validator("DateOfAuthorisationForIssueOfFinancialStatements", mode="before")
    def validate_authorisation_date(cls, v: str) -> str:
        return validate_date(v)

class DirectorsStatement(BaseModel):
    """Statements made by the directors regarding the financial statements"""
    WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView: bool = Field(..., description="Directors' opinion on whether financial statements give a true and fair view")
    WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement: bool = Field(..., description="Directors' opinion on solvency of the company")

class AuditReport(BaseModel):
    """Information about the independent auditors' report"""
    TypeOfAuditOpinionInIndependentAuditorsReport: AuditOpinion = Field(..., description="Type of opinion expressed by the auditors")
    AuditingStandardsUsedToConductTheAudit: Optional[str] = Field(None, description="Auditing standards framework used for the audit")
    WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern: Optional[bool] = Field(None, description="Whether auditors reported material uncertainty about going concern")
    WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKept: Optional[bool] = Field(None, description="Auditors' opinion on whether proper accounting records have been kept")

# Current Assets
class CurrentAssets(BaseModel):
    """Current assets section"""
    CashAndBankBalances: Optional[float] = Field(None,  description="Cash and bank balances, current. Common terms: cash, cash equivalents, bank balances, cash at bank, cash on hand")
    TradeAndOtherReceivablesCurrent: Optional[float] = Field(None,  description="Trade and other receivables (including contract assets), current. Common terms: accounts receivable, trade debtors, trade receivables")
    CurrentFinanceLeaseReceivables: Optional[float] = Field(None,  description="Financial assets - lease receivables, current. Common terms: lease receivables, finance lease assets, short-term lease receivables")
    CurrentDerivativeFinancialAssets: Optional[float] = Field(None,  description="Financial assets - derivatives, current. Common terms: derivative assets, forward contracts, swap assets, option assets")
    CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss: Optional[float] = Field(None,  description="Financial assets at fair value through profit or loss, current. Common terms: FVTPL assets, trading assets, fair value instruments")
    OtherCurrentFinancialAssets: Optional[int] = Field(None,  description="Other financial assets, current. Common terms: short-term investments, marketable securities, financial instruments")
    DevelopmentProperties: Optional[float] = Field(None,  description="Inventories - development properties, current. Common terms: property development, properties under development, development inventory")
    Inventories: Optional[float] = Field(None,  description="Inventories - others, current. Common terms: stock, goods for sale, raw materials, finished goods, work in progress")
    OtherCurrentNonfinancialAssets: Optional[float] = Field(None,  description="Other non-financial assets, current. Common terms: prepayments, advances, deposits, other current assets")
    NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners: Optional[float] = Field(None,  description="Non-current assets held for sale. Common terms: assets held for sale, disposal groups, discontinued operations assets")
    CurrentAssets: float = Field(...,  description="Total current assets. Common terms: total current assets, current assets, short-term assets")

# Non-Current Assets
class NonCurrentAssets(BaseModel):
    """Non-current assets section"""
    TradeAndOtherReceivablesNoncurrent: Optional[float] = Field(None,  description="Trade and other receivables, non-current. Common terms: long-term receivables, non-current debtors, long-term deposits")
    NoncurrentFinanceLeaseReceivables: Optional[float] = Field(None,  description="Financial assets - lease receivables, non-current. Common terms: long-term lease receivables, non-current leasing assets")
    NoncurrentDerivativeFinancialAssets: Optional[float] = Field(None,  description="Financial assets - derivatives, non-current. Common terms: long-term derivative assets, non-current hedging instruments")
    NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss: Optional[float] = Field(None,  description="Financial assets at fair value through P/L, non-current. Common terms: long-term FVTPL assets, non-current fair value assets")
    OtherNoncurrentFinancialAssets: Optional[float] = Field(None,  description="Other financial assets, non-current. Common terms: long-term investments, bonds held, held-to-maturity investments")
    PropertyPlantAndEquipment: Optional[float] = Field(None,  description="Property, plant and equipment. Common terms: PPE, fixed assets, tangible assets, property and equipment")
    InvestmentProperties: Optional[float] = Field(None,  description="Investment properties. Common terms: investment real estate, rental properties, property investments")
    Goodwill: Optional[float] = Field(None,  description="Goodwill. Common terms: acquisition goodwill, purchased goodwill, business combination goodwill")
    IntangibleAssetsOtherThanGoodwill: Optional[float] = Field(None,  description="Intangible assets excluding goodwill. Common terms: patents, trademarks, software, licenses, intellectual property")
    InvestmentsInSubsidiariesAssociatesOrJointVentures: Optional[float] = Field(None,  description="Investments in subsidiaries, associates and joint ventures. Common terms: equity investments, associated companies, joint ventures")
    DeferredTaxAssets: Optional[float] = Field(None,  description="Deferred tax assets. Common terms: future tax assets, tax credits, deferred taxation assets")
    OtherNoncurrentNonfinancialAssets: Optional[float] = Field(None,  description="Other non-financial assets, non-current. Common terms: long-term prepayments, non-current advances")
    NoncurrentAssets: float = Field(...,  description="Total non-current assets. Common terms: total non-current assets, long-term assets, fixed assets")

# Current Liabilities
class CurrentLiabilities(BaseModel):
    """Current liabilities section"""
    TradeAndOtherPayablesCurrent: Optional[float] = Field(None,  description="Trade and other payables, current. Common terms: accounts payable, trade creditors, accruals, trade payables")
    CurrentLoansAndBorrowings: Optional[float] = Field(None,  description="Loans and borrowings, current. Common terms: short-term borrowings, bank overdrafts, current portion of loans")
    CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss: Optional[float] = Field(None,  description="Financial liabilities at fair value through P/L, current. Common terms: current FVTPL liabilities, derivative liabilities")
    CurrentFinanceLeaseLiabilities: Optional[float] = Field(None,  description="Finance lease liabilities, current. Common terms: current lease obligations, short-term lease liabilities")
    OtherCurrentFinancialLiabilities: Optional[float] = Field(None,  description="Other financial liabilities, current. Common terms: other financial obligations, financial payables")
    CurrentIncomeTaxLiabilities: Optional[float] = Field(None,  description="Income tax liabilities, current. Common terms: tax payable, current tax, income tax payable")
    CurrentProvisions: Optional[float] = Field(None,  description="Provisions, current. Common terms: short-term provisions, current provisions, warranty provisions")
    OtherCurrentNonfinancialLiabilities: Optional[float] = Field(None,  description="Other non-financial liabilities, current. Common terms: deferred income, contract liabilities, advances received")
    LiabilitiesClassifiedAsHeldForSale: Optional[float] = Field(None,  description="Liabilities classified as held for sale. Common terms: disposal group liabilities, discontinued operations liabilities")
    CurrentLiabilities: float = Field(...,  description="Total current liabilities. Common terms: total current liabilities, short-term liabilities")

# Non-Current Liabilities
class NonCurrentLiabilities(BaseModel):
    """Non-current liabilities section"""
    TradeAndOtherPayablesNoncurrent: Optional[float] = Field(None,  description="Trade and other payables, non-current. Common terms: long-term payables, non-current creditors")
    NoncurrentLoansAndBorrowings: Optional[float] = Field(None,  description="Loans and borrowings, non-current. Common terms: long-term borrowings, term loans, bonds payable")
    NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss: Optional[float] = Field(None,  description="Financial liabilities at fair value through P/L, non-current. Common terms: long-term FVTPL liabilities, long-term derivatives")
    NoncurrentFinanceLeaseLiabilities: Optional[float] = Field(None,  description="Finance lease liabilities, non-current. Common terms: long-term lease obligations, long-term lease liabilities")
    OtherNoncurrentFinancialLiabilities: Optional[float] = Field(None,  description="Other financial liabilities, non-current. Common terms: other long-term financial obligations")
    DeferredTaxLiabilities: Optional[float] = Field(None,  description="Deferred tax liabilities. Common terms: future tax liabilities, deferred taxation")
    NoncurrentProvisions: Optional[float] = Field(None,  description="Provisions, non-current. Common terms: long-term provisions, decommissioning provisions, restoration provisions")
    OtherNoncurrentNonfinancialLiabilities: Optional[float] = Field(None,  description="Other non-financial liabilities, non-current. Common terms: long-term deferred income, long-term contract liabilities")
    NoncurrentLiabilities: float = Field(...,  description="Total non-current liabilities. Common terms: total non-current liabilities, long-term liabilities")

# Equity
class Equity(BaseModel):
    """Equity section"""
    ShareCapital: float = Field(...,  description="Share capital. Common terms: issued capital, paid-up capital, ordinary shares, common stock")
    TreasuryShares: Optional[float] = Field(None,  description="Treasury shares. Common terms: own shares, repurchased shares, treasury stock")
    AccumulatedProfitsLosses: float = Field(..., description="Accumulated profits or losses. Common terms: retained earnings, retained profits, accumulated earnings")
    ReservesOtherThanAccumulatedProfitsLosses: Optional[float] = Field(None, description="Other reserves. Common terms: revaluation reserve, translation reserve, hedging reserve, capital reserve")
    NoncontrollingInterests: Optional[float] = Field(None, description="Non-controlling interests. Common terms: minority interests, NCI, minority shareholders' interest")
    Equity: float = Field(..., description="Total equity. Common terms: shareholders' equity, shareholders' funds, net assets, total equity")

class StatementOfFinancialPosition(BaseModel):
    """Balance sheet information"""
    currentAssets: CurrentAssets
    nonCurrentAssets: NonCurrentAssets
    Assets: float = Field(...,  description="Total assets (CurrentAssets + NoncurrentAssets)")
    currentLiabilities: CurrentLiabilities
    nonCurrentLiabilities: NonCurrentLiabilities
    Liabilities: float = Field(...,  description="Total liabilities (CurrentLiabilities + NoncurrentLiabilities)")
    equity: Equity
    
    class Config:
        extra = "forbid"  # Equivalent to strict() in Zod

# Income Statement
class IncomeStatement(BaseModel):
    """Income statement information"""
    Revenue: float = Field(...,  description="Revenue from contracts with customers. Common terms: turnover, sales, contract revenue, income from operations")
    OtherIncome: Optional[float] = Field(None,  description="Other income not from primary operations. Common terms: other operating income, dividend income, interest income, rental income")
    EmployeeBenefitsExpense: Optional[float] = Field(None,  description="Employee benefits expense. Common terms: staff costs, salaries, wages, personnel expenses, CPF, compensation, bonuses")
    DepreciationExpense: Optional[float] = Field(None,  description="Depreciation of property, plant and equipment. Common terms: depreciation expense, PPE depreciation, fixed asset depreciation")
    AmortisationExpense: Optional[float] = Field(None,  description="Amortisation of intangible assets. Common terms: amortization expense, goodwill amortization, intangible amortization")
    RepairsAndMaintenanceExpense: Optional[float] = Field(None,  description="Repairs and maintenance costs. Common terms: upkeep expenses, maintenance costs, repair expenses")
    SalesAndMarketingExpense: Optional[float] = Field(None,  description="Sales and marketing costs. Common terms: marketing expenses, advertising, selling expenses, promotion, distribution costs")
    OtherExpensesByNature: Optional[float] = Field(None,  description="Other operating expenses by nature. Common terms: general expenses, administrative costs, utilities, office expenses, rental expenses")
    OtherGainsLosses: Optional[float] = Field(None, description="Other gains/(losses). Common terms: foreign exchange gains/losses, forex, fair value gains/losses, disposal gains/losses")
    FinanceCosts: Optional[float] = Field(None,  description="Net finance costs. Common terms: interest expense, borrowing costs, loan interest, financing costs")
    ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod: Optional[float] = Field(None, description="Share of profits/(losses) of associates/joint ventures. Common terms: equity method investments, associate profits, joint venture results")
    ProfitLossBeforeTaxation: float = Field(..., description="Profit/(loss) before tax from continuing operations. Common terms: profit before tax, earnings before tax, EBT, PBT")
    TaxExpenseBenefitContinuingOperations: float = Field(..., description="Income tax expense/(benefit). Common terms: tax expense, taxation, income tax, tax charge")
    ProfitLossFromDiscontinuedOperations: Optional[float] = Field(None, description="Profit/(loss) from discontinued operations. Common terms: discontinued operations, disposal group results")
    ProfitLoss: float = Field(..., description="Total comprehensive income for the period. Common terms: profit for the year, net profit, profit for the period, net income")
    ProfitLossAttributableToOwnersOfCompany: float = Field(..., description="Portion attributable to parent owners. Common terms: attributable to shareholders, equity holders, owners of the company")
    ProfitLossAttributableToNoncontrollingInterests: Optional[float] = Field(None, description="Portion attributable to NCI. Common terms: non-controlling interests, minority interest, minority shareholders")
    
    class Config:
        extra = "forbid"  # Equivalent to strict() in Zod

# Trade and Other Receivables
class TradeAndOtherReceivables(BaseModel):
    """Trade and other receivables detail"""
    TradeAndOtherReceivablesDueFromThirdParties: Optional[float] = Field(None,  description="Receivables from third parties. Common terms: third party receivables, external receivables")
    TradeAndOtherReceivablesDueFromRelatedParties: Optional[float] = Field(None,  description="Receivables from related parties. Common terms: related party receivables, intercompany receivables")
    UnbilledReceivables: Optional[float] = Field(None,  description="Unbilled receivables. Common terms: contract assets, accrued income, work completed not billed")
    OtherReceivables: Optional[float] = Field(None,  description="Other receivables. Common terms: sundry receivables, miscellaneous receivables, deposits")
    TradeAndOtherReceivables: float = Field(...,  description="Total trade and other receivables. Common terms: accounts receivable, debtors, total receivables")

# Trade and Other Payables
class TradeAndOtherPayables(BaseModel):
    """Trade and other payables detail"""
    TradeAndOtherPayablesDueToThirdParties: Optional[float] = Field(None,  description="Payables to third parties. Common terms: third party payables, external payables")
    TradeAndOtherPayablesDueToRelatedParties: Optional[float] = Field(None,  description="Payables to related parties. Common terms: related party payables, intercompany payables")
    DeferredIncome: Optional[float] = Field(None,  description="Deferred income. Common terms: unearned revenue, contract liabilities, advances from customers")
    OtherPayables: Optional[float] = Field(None,  description="Other payables. Common terms: sundry payables, accruals, other creditors")
    TradeAndOtherPayables: float = Field(...,  description="Total trade and other payables. Common terms: accounts payable, creditors, total payables")
    
    class Config:
        extra = "forbid"  # Equivalent to strict() in Zod

# Revenue
class Revenue(BaseModel):
    """Revenue detail"""
    RevenueFromPropertyTransferredAtPointInTime: Optional[float] = Field(None,  description="Revenue from property at point in time. Common terms: property sales, real estate sales")
    RevenueFromGoodsTransferredAtPointInTime: Optional[float] = Field(None,  description="Revenue from goods at point in time. Common terms: product sales, goods sold, merchandise sales")
    RevenueFromServicesTransferredAtPointInTime: Optional[float] = Field(None,  description="Revenue from services at point in time. Common terms: service fees, one-time services")
    RevenueFromPropertyTransferredOverTime: Optional[float] = Field(None,  description="Revenue from property over time. Common terms: development revenue, long-term property projects")
    RevenueFromConstructionContractsOverTime: Optional[float] = Field(None,  description="Revenue from construction contracts over time. Common terms: construction revenue, contract revenue")
    RevenueFromServicesTransferredOverTime: Optional[float] = Field(None,  description="Revenue from services over time. Common terms: recurring services, subscription revenue, ongoing services")
    OtherRevenue: Optional[float] = Field(None,  description="Other revenue. Common terms: miscellaneous revenue, other income streams")
    Revenue: float = Field(...,  description="Total revenue. Common terms: total revenue, turnover, sales, total income")
    
    class Config:
        extra = "forbid"  # Equivalent to strict() in Zod

class Notes(BaseModel):
    """Notes to financial statements"""
    tradeAndOtherReceivables: TradeAndOtherReceivables
    tradeAndOtherPayables: TradeAndOtherPayables
    revenue: Revenue
    
    class Config:
        title = "Comprehensive financial statement schema compliant with Singapore Simplified XBRL requirements"

class PartialXBRL(BaseModel):
    """Singapore XBRL schema"""
    filingInformation: FilingInformation
    directorsStatement: DirectorsStatement
    auditReport: AuditReport
    statementOfFinancialPosition: StatementOfFinancialPosition
    incomeStatement: IncomeStatement
    notes: Notes

### SYSTEM PROMPT

In [ ]:
# Mapping System Prompt
FINANCIAL_STATEMENT_PROMPT = """You are a Singapore financial reporting specialist who converts annual reports to XBRL format.
Your task is to extract and map financial data from input reports into standardized Statement of Profit or Loss and Statement of Financial Position models.

## AVAILABLE TOOLS AND DEPENDENCIES

1. `match_financial_term`: This tool helps identify which standardized field a term from the financial report maps to.
   - Input: Any financial term from the report
   - Output: The standardized field name, statement type, and match confidence
   - Uses the FinancialTermDeps dependency which contains pre-defined term mappings
   - If a term is not found in the dependencies, the tool will use accounting knowledge to make a best guess

2. `process_financial_data`: This tool extracts and categorizes values from nested data structures.
   - Input: A nested financial data structure
   - Output: Organized values by statement type (income_statement, financial_position)

## DEPENDENCIES

1. `FinancialTermDeps`: Contains comprehensive mappings for financial statement terms
   - Income statement terms: Standard mappings for revenue, expenses, profits, etc.
   - Financial position terms: Standard mappings for assets, liabilities, equity, etc.
   - The tool uses these mappings to correctly identify and categorize financial data

## MAPPING PROCESS

1. ANALYZE the input data:
   - Identify the overall structure of the financial data
   - Determine which sections contain income statement vs. financial position data
   - Check for both directly accessible and nested financial values

2. EXTRACT values systematically:
   - Use the process_financial_data tool to process complex structures
   - For simple structures, map fields directly using match_financial_term

3. VALIDATE the extracted data:
   - Ensure all required fields are populated
   - Verify that calculations are consistent (e.g., gross profit = revenue - cost of sales)
   - Check that assets = liabilities + equity in the financial position statement

4. STANDARDIZE values:
   - Ensure expenses are consistently represented (typically as negative values)
   - Normalize any unusual formats or units
   - Handle any currency conversions needed

## HANDLING UNKNOWN TERMS

If `match_financial_term` encounters a term not found in the dependencies:
1. It will apply accounting knowledge to determine the most appropriate classification
2. Look for contextual clues (e.g., location in statement, relationship to other items)
3. Use common Singapore financial reporting standards as a guide
4. Make a best effort match based on the term's semantic meaning

## IMPORTANT GUIDELINES

1. COMPREHENSIVE MAPPING: Identify and map ALL financial fields in the input data
   - Search through all sections, including nested structures
   - Pay special attention to owner/non-controlling breakdowns
   - Map both primary items and subtotals

2. SEMANTIC MAPPING: Focus on accounting meaning, not just exact wording
   - Consider Singapore-specific financial terminology
   - Use contextual clues to determine proper classification
   - Use dependencies first, then fall back to general accounting knowledge

3. HANDLING EXPENSES: Ensure consistent sign convention
   - Typically represent expenses as negative values
   - Convert positive expense values to negative if needed

4. FINANCIAL POSITION SPECIFICS:
   - Categorize assets into current and non-current
   - Categorize liabilities into current and non-current
   - Ensure equity components are properly identified
   - Verify the balance sheet equation (Assets = Liabilities + Equity)

Be thorough, precise, and follow Singapore accounting standards in your mappings.
"""

### DEPENDENCIES

In [ ]:
# Mapping dependencies
@dataclass
class FinancialTermDeps:
    """Dependencies for financial term mapping"""
    income_statement_terms: Dict[str, List[str]]
    financial_position_terms: Dict[str, List[str]]
    
    def __post_init__(self):
        """Ensure all terms are lowercase for case-insensitive matching"""
        self.income_statement_terms = {
            k: [t.lower() for t in terms] for k, terms in self.income_statement_terms.items()
        }
        self.financial_position_terms = {
            k: [t.lower() for t in terms] for k, terms in self.financial_position_terms.items()
        }

# Create term mappings
income_terms = {
    "Revenue": ["revenue", "turnover", "sales", "contract", "income from operations"],
    "OtherIncome": ["other income", "other operating income", "other revenue", "dividend income", "interest income", "rental income"],
    "EmployeeBenefitsExpense": ["employee", "staff", "personnel", "salaries", "wages", "cpf", "compensation", "bonuses"],
    "DepreciationExpense": ["depreciation", "property, plant and equipment", "ppe", "fixed assets", "depreciate"],
    "AmortisationExpense": ["amortisation", "amortization", "intangible assets", "goodwill", "amortize"],
    "RepairsAndMaintenanceExpense": ["repairs", "maintenance", "upkeep", "repair expenses", "maintenance costs"],
    "SalesAndMarketingExpense": ["sales", "marketing", "advertising", "promotion", "selling expenses", "distribution costs"],
    "OtherExpensesByNature": ["other expenses", "general expenses", "administrative", "utilities", "office expenses", "rental expenses"],
    "OtherGainsLosses": ["other gains", "other losses", "foreign exchange", "forex", "fair value", "disposal gains", "disposal losses"],
    "FinanceCosts": ["finance costs", "interest expense", "borrowing costs", "loan interest", "financing costs"],
    "ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod": ["associates", "joint ventures", "equity method", "share of profit", "share of loss"],
    "ProfitLossBeforeTaxation": ["profit before tax", "earnings before tax", "profit before income tax", "pbt", "ebt"],
    "TaxExpenseBenefitContinuingOperations": ["income tax", "tax expense", "taxation", "tax charge", "income tax expense"],
    "ProfitLossFromDiscontinuedOperations": ["discontinued operations", "disposal group", "discontinued business"],
    "ProfitLoss": ["profit for the year", "net profit", "profit for the period", "net income", "total comprehensive income"],
    "ProfitLossAttributableToOwnersOfCompany": ["attributable to owners", "attributable to shareholders", "equity holders", "owners of the company"],
    "ProfitLossAttributableToNoncontrollingInterests": ["non-controlling interests", "minority interest", "minority shareholders", "nci"]
}

position_terms = {
    "currentAssets.CashAndBankBalances": ["cash", "bank", "cash equivalents", "cash at bank", "cash on hand"],
    "currentAssets.TradeAndOtherReceivablesCurrent": ["trade receivables", "accounts receivable", "trade debtors", "contract assets", "current receivables"],
    "currentAssets.CurrentFinanceLeaseReceivables": ["lease receivables", "finance lease assets", "short-term lease receivables"],
    "currentAssets.CurrentDerivativeFinancialAssets": ["derivatives", "forward contracts", "swap assets", "option assets", "current derivative assets"],
    "currentAssets.CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss": ["fvtpl", "fair value", "trading assets", "fair value instruments", "current financial assets at fair value"],
    "currentAssets.Inventories": ["inventory", "stock", "goods for sale", "raw materials", "finished goods", "work in progress"],
    "currentAssets.OtherCurrentNonfinancialAssets": ["prepayments", "advances", "deposits", "other current assets"],
    "currentAssets.CurrentAssets": ["total current assets", "current assets", "short-term assets"],
    
    "nonCurrentAssets.PropertyPlantAndEquipment": ["ppe", "fixed assets", "tangible assets", "property and equipment", "plant and equipment"],
    "nonCurrentAssets.InvestmentProperties": ["investment property", "investment real estate", "rental properties", "property investments"],
    "nonCurrentAssets.Goodwill": ["goodwill", "acquisition goodwill", "purchased goodwill"],
    "nonCurrentAssets.IntangibleAssetsOtherThanGoodwill": ["intangible assets", "patents", "trademarks", "software", "licenses", "intellectual property"],
    "nonCurrentAssets.InvestmentsInSubsidiariesAssociatesOrJointVentures": ["investments in subsidiaries", "investments in associates", "equity investments", "joint ventures"],
    "nonCurrentAssets.DeferredTaxAssets": ["deferred tax assets", "future tax assets", "tax credits"],
    "nonCurrentAssets.NoncurrentAssets": ["total non-current assets", "long-term assets", "fixed assets"],
    
    "currentLiabilities.TradeAndOtherPayablesCurrent": ["trade payables", "accounts payable", "trade creditors", "accruals", "current payables"],
    "currentLiabilities.CurrentLoansAndBorrowings": ["short-term borrowings", "bank overdrafts", "current portion of loans", "current loans"],
    "currentLiabilities.CurrentFinanceLeaseLiabilities": ["current lease obligations", "short-term lease liabilities", "lease liabilities current"],
    "currentLiabilities.CurrentIncomeTaxLiabilities": ["tax payable", "current tax", "income tax payable"],
    "currentLiabilities.CurrentProvisions": ["short-term provisions", "current provisions", "warranty provisions"],
    "currentLiabilities.CurrentLiabilities": ["total current liabilities", "short-term liabilities"],
    
    "nonCurrentLiabilities.NoncurrentLoansAndBorrowings": ["long-term borrowings", "term loans", "bonds payable", "non-current loans"],
    "nonCurrentLiabilities.NoncurrentFinanceLeaseLiabilities": ["long-term lease obligations", "long-term lease liabilities", "non-current lease liabilities"],
    "nonCurrentLiabilities.DeferredTaxLiabilities": ["deferred tax liabilities", "future tax liabilities", "deferred taxation"],
    "nonCurrentLiabilities.NoncurrentProvisions": ["long-term provisions", "decommissioning provisions", "restoration provisions"],
    "nonCurrentLiabilities.NoncurrentLiabilities": ["total non-current liabilities", "long-term liabilities"],
    
    "equity.ShareCapital": ["share capital", "issued capital", "paid-up capital", "ordinary shares", "common stock"],
    "equity.TreasuryShares": ["treasury shares", "own shares", "repurchased shares", "treasury stock"],
    "equity.AccumulatedProfitsLosses": ["retained earnings", "accumulated profits", "retained profits", "accumulated earnings"],
    "equity.ReservesOtherThanAccumulatedProfitsLosses": ["reserves", "revaluation reserve", "translation reserve", "hedging reserve", "capital reserve"],
    "equity.NoncontrollingInterests": ["minority interests", "nci", "minority shareholders' interest", "non-controlling interests"],
    "equity.Equity": ["total equity", "shareholders' equity", "shareholders' funds", "net assets"],
    
    "Assets": ["total assets", "assets"],
    "Liabilities": ["total liabilities", "liabilities"]
}

### AGENT

In [ ]:
# SETUP MAPPING AGENT

mapping_model = OpenAIModel(model_name="gpt-4o", api_key=OPENAI_API_KEY)

financial_deps = FinancialTermDeps(income_terms, position_terms)

# Define the agent with dependencies
financial_statement_agent = Agent(
    model=mapping_model,
    result_type=PartialXBRL,
    system_prompt=FINANCIAL_STATEMENT_PROMPT,
    deps_type=financial_deps,
    retries=3
)

### TOOLS

In [ ]:
# Mappping match_financial_term tool
class MatchTermContext(BaseModel):
    """Context for the term matching tool"""
    pass

@financial_statement_agent.tool
def match_financial_term(context: RunContext[FinancialTermDeps], term: str, statement_type: str = "all") -> Dict[str, Any]:
    """
    Match a financial term to the standardized field name in financial statements.
    Uses injected financial term mappings from dependencies.
    
    Args:
        context: The tool context with injected dependencies
        term: The financial term to match (e.g., "Revenue", "Staff Costs", "Cash and bank", etc.)
        statement_type: Type of statement to match against ("income", "position", "all")
        
    Returns:
        Dictionary with matching field name, statement type, and confidence level
    """
    term_lower = term.lower().strip()
    
    # Get mappings from dependencies
    income_mappings = context.deps.income_statement_terms
    position_mappings = context.deps.financial_position_terms
    
    # Filter mappings based on statement_type
    filtered_mappings = {}
    
    if statement_type.lower() in ["all", "income", "profit", "loss"]:
        for field, keywords in income_mappings.items():
            filtered_mappings[f"income.{field}"] = keywords
    
    if statement_type.lower() in ["all", "position", "balance", "financial_position"]:
        for field, keywords in position_mappings.items():
            filtered_mappings[f"position.{field}"] = keywords
    
    # If no valid statement type provided, use all mappings
    if not filtered_mappings:
        for field, keywords in income_mappings.items():
            filtered_mappings[f"income.{field}"] = keywords
        for field, keywords in position_mappings.items():
            filtered_mappings[f"position.{field}"] = keywords
    
    # Find matches
    matches = {}
    for full_field, keywords in filtered_mappings.items():
        score = 0
        for keyword in keywords:
            if keyword in term_lower:
                score += 1
            # Extra points for exact matches or close matches
            if keyword == term_lower:
                score += 5
            elif keyword in term_lower and len(keyword) > 5:
                score += 2
                
        if score > 0:
            matches[full_field] = score
    
    # If we have matches, return the best one
    if matches:
        best_match = max(matches.items(), key=lambda x: x[1])
        statement_type, field_name = best_match[0].split('.', 1)
        
        return {
            "statement_type": "income_statement" if statement_type == "income" else "financial_position",
            "field": field_name,
            "match_score": best_match[1],
            "matched_term": term
        }
    
    # No matches found - make a best guess based on the term
    if any(word in term_lower for word in ["revenue", "income", "sale", "expense", "cost", "profit", "loss", "tax"]):
        return {
            "statement_type": "income_statement",
            "field": term,
            "match_score": 0,
            "matched_term": term
        }
    elif any(word in term_lower for word in ["asset", "liability", "equity", "cash", "receivable", "payable", "property", "equipment"]):
        return {
            "statement_type": "financial_position", 
            "field": term,
            "match_score": 0,
            "matched_term": term
        }
    
    # Truly unknown
    return {
        "statement_type": "unknown",
        "field": "unknown",
        "match_score": 0,
        "matched_term": term
    }

In [ ]:
# Mapping extract_and_categorize_financial_data tool
from typing import Union

# Define the simplified extraction tool
class FinancialData(BaseModel):
    """Schema for financial data structure"""
    data: Dict[str, Any] = Field(..., description="Financial data items")
    
    # Alternative constructor for direct dictionary input
    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        return cls(data=data)

class ExtractContext(BaseModel):
    """Context for the extraction tool"""
    pass

@financial_statement_agent.tool
def extract_and_categorize_financial_data(context: RunContext[ExtractContext], data: Union[Dict[str, Any], FinancialData], field_path: str = "") -> Dict[str, Dict[str, float]]:
    """
    Extract and categorize financial values from nested structures, organizing them into appropriate statement models.
    Handles direct values without CFY/PFY structure, assuming each field has only one value.
    
    Args:
        context: The tool context
        data: Financial data structure (raw dictionary or FinancialData object)
        field_path: Current path in the nested structure (used in recursion)
        
    Returns:
        Dictionary with categorized financial data organized by statement type and field
    """
    # Initialize results with statement types
    results = {
        "income_statement": {},
        "financial_position": {},
        "unknown": {}
    }
    
    # Handle different input types
    if isinstance(data, dict):
        items_dict = data
    elif hasattr(data, 'data'):
        items_dict = data.data
    else:
        items_dict = {}
        
    # Special case: Check if data is already in statement form
    if "incomeStatement" in items_dict and isinstance(items_dict["incomeStatement"], dict):
        results["income_statement"] = {k: float(v) for k, v in items_dict["incomeStatement"].items() if isinstance(v, (int, float))}
    
    if "statementOfFinancialPosition" in items_dict and isinstance(items_dict["statementOfFinancialPosition"], dict):
        # Extract values from the financial position structure
        financial_position = items_dict["statementOfFinancialPosition"]
        
        # First handle top-level items
        for key, value in financial_position.items():
            if isinstance(value, (int, float)):
                results["financial_position"][key] = float(value)
        
        # Handle nested structures (current assets, non-current assets, etc.)
        for section in ["currentAssets", "nonCurrentAssets", "currentLiabilities", "nonCurrentLiabilities", "equity"]:
            if section in financial_position and isinstance(financial_position[section], dict):
                for key, value in financial_position[section].items():
                    if isinstance(value, (int, float)):
                        results["financial_position"][f"{section}.{key}"] = float(value)
    
    # Process regular dictionary structure
    for key, value in items_dict.items():
        # Skip already processed statement sections
        if key in ["incomeStatement", "statementOfFinancialPosition"]:
            continue
            
        current_path = f"{field_path}_{key}" if field_path else key
        
        # Process direct numeric values
        if isinstance(value, (int, float)):
            float_value = float(value)
            term_info = match_financial_term(context, current_path)
            
            if term_info["statement_type"] != "unknown":
                results[term_info["statement_type"]][term_info["field"]] = float_value
            else:
                term_info_key_only = match_financial_term(context, key)
                if term_info_key_only["statement_type"] != "unknown":
                    results[term_info_key_only["statement_type"]][term_info_key_only["field"]] = float_value
                else:
                    results["unknown"][current_path] = float_value
        
        # Process nested dictionaries
        elif isinstance(value, dict):
            if len(value) == 1 and isinstance(list(value.values())[0], (int, float)):
                term_info = match_financial_term(context, current_path)
                if term_info["statement_type"] != "unknown":
                    results[term_info["statement_type"]][term_info["field"]] = float(list(value.values())[0])
                else:
                    results["unknown"][current_path] = float(list(value.values())[0])
                continue
            
            nested_results = extract_and_categorize_financial_data(context, value, current_path)
            for statement_type, fields in nested_results.items():
                for field, value in fields.items():
                    if statement_type in results:
                        results[statement_type][field] = value
        
        # Handle arrays/lists
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    list_path = f"{current_path}[{i}]"
                    nested_results = extract_and_categorize_financial_data(context, item, list_path)
                    
                    for statement_type, fields in nested_results.items():
                        for field, value in fields.items():
                            if statement_type in results:
                                results[statement_type][field] = value
                elif isinstance(item, (int, float)) and i == 0:
                    term_info = match_financial_term(context, current_path)
                    if term_info["statement_type"] != "unknown":
                        results[term_info["statement_type"]][term_info["field"]] = float(item)
                    else:
                        results["unknown"][current_path] = float(item)
    
    # Remove empty statement types
    for statement_type in list(results.keys()):
        if not results[statement_type]:
            del results[statement_type]
    
    return results

### DUMMY DATA

In [ ]:
dummy_data = {
  "filingInformation": {
    "NameOfCompany": "ACME Corporation",
    "UniqueEntityNumber": "123456789A",
    "CurrentPeriodStartDate": "2022-01-01",
    "CurrentPeriodEndDate": "2022-12-31",
    "PriorPeriodStartDate": "2021-01-01",
    "TypeOfXBRLFiling": "Full",
    "NatureOfFinancialStatementsCompanyLevelOrConsolidated": "Company",
    "TypeOfAccountingStandardUsedToPrepareFinancialStatements": "IFRS",
    "DateOfAuthorisationForIssueOfFinancialStatements": "2023-03-15",
    "TypeOfStatementOfFinancialPosition": "Classified",
    "WhetherTheFinancialStatementsArePreparedOnGoingConcernBasis": True,
    "WhetherThereAreAnyChangesToComparativeAmounts": False,
    "DescriptionOfPresentationCurrency": "USD",
    "DescriptionOfFunctionalCurrency": "USD",
    "LevelOfRoundingUsedInFinancialStatements": "Units",
    "DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities": "Manufacturing and distribution of consumer electronics.",
    "PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice": "123 Business Street, Industrial Park",
    "WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees": True,
    "NameOfParentEntity": None,
    "NameOfUltimateParentOfGroup": None,
    "TaxonomyVersion": "2022.2",
    "NameAndVersionOfSoftwareUsedToGenerateXBRLFile": "XBRL Generator v1.0",
    "HowWasXBRLFilePrepared": "Automated"
  },
  "directorsStatement": {
    "WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView": True,
    "WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement": True
  },
  "auditReport": {
    "TypeOfAuditOpinionInIndependentAuditorsReport": "Unqualified",
    "AuditingStandardsUsedToConductTheAudit": "ISA",
    "WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern": False,
    "WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKept": True
  },
  "statementOfFinancialPosition": {
    "currentAssets": {
      "CashAndBankBalances": 150000,
      "TradeAndOtherReceivablesCurrent": 300000,
      "CurrentFinanceLeaseReceivables": 20000,
      "CurrentDerivativeFinancialAssets": 5000,
      "CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss": 10000,
      "OtherCurrentFinancialAssets": 0,
      "DevelopmentProperties": 0,
      "Inventories": 45000,
      "OtherCurrentNonfinancialAssets": 0,
      "NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners": 0,
      "CurrentAssets": 500000
    },
    "nonCurrentAssets": {
      "TradeAndOtherReceivablesNoncurrent": 200000,
      "NoncurrentFinanceLeaseReceivables": 15000,
      "NoncurrentDerivativeFinancialAssets": 7000,
      "NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss": 12000,
      "OtherNoncurrentFinancialAssets": 0,
      "PropertyPlantAndEquipment": 800000,
      "InvestmentProperties": 50000,
      "Goodwill": 30000,
      "IntangibleAssetsOtherThanGoodwill": 25000,
      "InvestmentsInSubsidiariesAssociatesOrJointVentures": 100000,
      "DeferredTaxAssets": 15000,
      "OtherNoncurrentNonfinancialAssets": 0,
      "NoncurrentAssets": 1200000
    },
    "Assets": 1700000,
    "currentLiabilities": {
      "TradeAndOtherPayablesCurrent": 100000,
      "CurrentLoansAndBorrowings": 50000,
      "CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss": 8000,
      "CurrentFinanceLeaseLiabilities": 6000,
      "OtherCurrentFinancialLiabilities": 0,
      "CurrentIncomeTaxLiabilities": 20000,
      "CurrentProvisions": 10000,
      "OtherCurrentNonfinancialLiabilities": 5000,
      "LiabilitiesClassifiedAsHeldForSale": 0,
      "CurrentLiabilities": 200000
    },
    "nonCurrentLiabilities": {
      "TradeAndOtherPayablesNoncurrent": 40000,
      "NoncurrentLoansAndBorrowings": 60000,
      "NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss": 5000,
      "NoncurrentFinanceLeaseLiabilities": 4000,
      "OtherNoncurrentFinancialLiabilities": 0,
      "DeferredTaxLiabilities": 15000,
      "NoncurrentProvisions": 7000,
      "OtherNoncurrentNonfinancialLiabilities": 0,
      "NoncurrentLiabilities": 200000
    },
    "Liabilities": 400000,
    "equity": {
      "ShareCapital": 500000,
      "TreasuryShares": 10000,
      "AccumulatedProfitsLosses": 300000,
      "ReservesOtherThanAccumulatedProfitsLosses": 50000,
      "NoncontrollingInterests": 0,
      "Equity": 790000
    }
  },
  "incomeStatement": {
    "Revenue": 1000000,
    "OtherIncome": 50000,
    "EmployeeBenefitsExpense": 200000,
    "DepreciationExpense": 50000,
    "AmortisationExpense": 10000,
    "RepairsAndMaintenanceExpense": 15000,
    "SalesAndMarketingExpense": 25000,
    "OtherExpensesByNature": 10000,
    "OtherGainsLosses": 0,
    "FinanceCosts": 8000,
    "ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod": 0,
    "ProfitLossBeforeTaxation": 150000,
    "TaxExpenseBenefitContinuingOperations": 30000,
    "ProfitLossFromDiscontinuedOperations": 0,
    "ProfitLoss": 120000,
    "ProfitLossAttributableToOwnersOfCompany": 110000,
    "ProfitLossAttributableToNoncontrollingInterests": 10000
  },
  "notes": {
    "tradeAndOtherReceivables": {
      "TradeAndOtherReceivablesDueFromThirdParties": 25000,
      "TradeAndOtherReceivablesDueFromRelatedParties": 15000,
      "UnbilledReceivables": 5000,
      "OtherReceivables": 2000,
      "TradeAndOtherReceivables": 45000
    },
    "tradeAndOtherPayables": {
      "TradeAndOtherPayablesDueToThirdParties": 20000,
      "TradeAndOtherPayablesDueToRelatedParties": 10000,
      "DeferredIncome": 3000,
      "OtherPayables": 2000,
      "TradeAndOtherPayables": 55000
    },
    "revenue": {
      "RevenueFromPropertyTransferredAtPointInTime": 0,
      "RevenueFromGoodsTransferredAtPointInTime": 0,
      "RevenueFromServicesTransferredAtPointInTime": 0,
      "RevenueFromPropertyTransferredOverTime": 5000,
      "RevenueFromConstructionContractsOverTime": 3000,
      "RevenueFromServicesTransferredOverTime": 7000,
      "OtherRevenue": 1000,
      "Revenue": 120000
    }
  }
}

In [ ]:
dummy_data_noise = {
  "filingInformation": {
    "CompanyName": "ACME Corporation",  # Changed from 'NameOfCompany'
    "UniqueEntityNumber": "123456789A",
    "CurrentPeriodStartDate": "2022-01-01",
    "CurrentPeriodEndDate": "2022-12-31",
    "PriorPeriodStartDate": "2021-01-01",
    "FilingType": "Full",  # Changed from 'TypeOfXBRLFiling'
    "NatureOfFinancialStatements": "Company",  # Changed from 'NatureOfFinancialStatementsCompanyLevelOrConsolidated'
    "AccountingStandardUsed": "IFRS",  # Changed from 'TypeOfAccountingStandardUsedToPrepareFinancialStatements'
    "DateOfAuthorisationForIssueOfFinancialStatements": "2023-03-15",
    "StatementOfFinancialPositionType": "Classified",  # Changed from 'TypeOfStatementOfFinancialPosition'
    "IsGoingConcernBasis": True,  # Changed from 'WhetherTheFinancialStatementsArePreparedOnGoingConcernBasis'
    "AreComparativeAmountsChanged": False,  # Changed from 'WhetherThereAreAnyChangesToComparativeAmounts'
    "PresentationCurrency": "USD",  # Changed from 'DescriptionOfPresentationCurrency'
    "FunctionalCurrency": "USD",  # Changed from 'DescriptionOfFunctionalCurrency'
    "RoundingLevel": "Units",  # Changed from 'LevelOfRoundingUsedInFinancialStatements'
    "NatureOfOperations": "Manufacturing and distribution of consumer electronics.",  # Changed from 'DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities'
    "BusinessAddress": "123 Business Street, Industrial Park",  # Changed from 'PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice'
    "HasMoreThan50Employees": True,  # Changed from 'WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees'
    "ParentEntityName": None,  # Changed from 'NameOfParentEntity'
    "UltimateParentEntityName": None,  # Changed from 'NameOfUltimateParentOfGroup'
    "TaxonomyVersion": "2022.2",
    "SoftwareUsed": "XBRL Generator v1.0",  # Changed from 'NameAndVersionOfSoftwareUsedToGenerateXBRLFile'
    "XBRLPreparationMethod": "Automated"  # Changed from 'HowWasXBRLFilePrepared'
  },
  "directorsStatement": {
    "IsTrueAndFairView": True,  # Changed from 'WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView'
    "CanPayDebtsWhenDue": True  # Changed from 'WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement'
  },
  "auditReport": {
    "AuditOpinionType": "Unqualified",  # Changed from 'TypeOfAuditOpinionInIndependentAuditorsReport'
    "AuditingStandardsUsed": "ISA",  # Changed from 'AuditingStandardsUsedToConductTheAudit'
    "IsGoingConcernUncertain": False,  # Changed from 'WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern'
    "AreRecordsProperlyKept": True  # Changed from 'WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKept'
  },
  "statementOfFinancialPosition": {
    "currentAssets": {
      "CashAndBankBalances": 150000,
      "TradeAndOtherReceivablesCurrent": 300000,
      "CurrentFinanceLeaseReceivables": 20000,
      "CurrentDerivativeFinancialAssets": 5000,
      "CurrentFinancialAssetsAtFVTPL": 10000,  # Changed from 'CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss'
      "OtherCurrentFinancialAssets": 0,
      "DevelopmentProperties": 0,
      "Inventories": 45000,
      "OtherCurrentNonfinancialAssets": 0,
      "AssetsHeldForSale": 0,  # Changed from 'NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners'
      "TotalCurrentAssets": 500000  # Changed from 'CurrentAssets'
    },
    "nonCurrentAssets": {
      "TradeAndOtherReceivablesNoncurrent": 200000,
      "NoncurrentFinanceLeaseReceivables": 15000,
      "NoncurrentDerivativeFinancialAssets": 7000,
      "NoncurrentFinancialAssetsAtFVTPL": 12000,  # Changed from 'NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss'
      "OtherNoncurrentFinancialAssets": 0,
      "PropertyPlantAndEquipment": 800000,
      "InvestmentProperties": 50000,
      "Goodwill": 30000,
      "OtherIntangibleAssets": 25000,  # Changed from 'IntangibleAssetsOtherThanGoodwill'
      "InvestmentsInSubsidiariesAssociatesOrJointVentures": 100000,
      "DeferredTaxAssets": 15000,
      "OtherNoncurrentNonfinancialAssets": 0,
      "TotalNoncurrentAssets": 1200000  # Changed from 'NoncurrentAssets'
    },
    "TotalAssets": 1700000,  # Changed from 'Assets'
    "currentLiabilities": {
      "TradeAndOtherPayablesCurrent": 100000,
      "CurrentLoansAndBorrowings": 50000,
      "CurrentFinancialLiabilitiesAtFVTPL": 8000,  # Changed from 'CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss'
      "CurrentFinanceLeaseLiabilities": 6000,
      "OtherCurrentFinancialLiabilities": 0,
      "CurrentIncomeTaxLiabilities": 20000,
      "CurrentProvisions": 10000,
      "OtherCurrentNonfinancialLiabilities": 5000,
      "LiabilitiesHeldForSale": 0,  # Changed from 'LiabilitiesClassifiedAsHeldForSale'
      "TotalCurrentLiabilities": 200000  # Changed from 'CurrentLiabilities'
    },
    "nonCurrentLiabilities": {
      "TradeAndOtherPayablesNoncurrent": 40000,
      "NoncurrentLoansAndBorrowings": 60000,
      "NoncurrentFinancialLiabilitiesAtFVTPL": 5000,  # Changed from 'NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss'
      "NoncurrentFinanceLeaseLiabilities": 4000,
      "OtherNoncurrentFinancialLiabilities": 0,
      "DeferredTaxLiabilities": 15000,
      "NoncurrentProvisions": 7000,
      "OtherNoncurrentNonfinancialLiabilities": 0,
      "TotalNoncurrentLiabilities": 200000  # Changed from 'NoncurrentLiabilities'
    },
    "TotalLiabilities": 400000,  # Changed from 'Liabilities'
    "equity": {
      "ShareCapital": 500000,
      "TreasuryShares": 10000,
      "RetainedEarnings": 300000,  # Changed from 'AccumulatedProfitsLosses'
      "OtherReserves": 50000,  # Changed from 'ReservesOtherThanAccumulatedProfitsLosses'
      "NonControllingInterests": 0,  # Changed from 'NoncontrollingInterests'
      "TotalEquity": 790000  # Changed from 'Equity'
    }
  },
  "incomeStatement": {
    "Revenue": 1000000,
    "OtherIncome": 50000,
    "EmployeeBenefitsExpense": 200000,
    "DepreciationExpense": 50000,
    "AmortizationExpense": 10000,  # Changed from 'AmortisationExpense'
    "ProfitOrLossFromDiscontinuedOperations": 0,
    "NetProfitOrLoss": 120000,  # Changed from 'ProfitLoss'
    "ProfitOrLossAttributableToOwners": 110000,  # Changed from 'ProfitLossAttributableToOwnersOfCompany'
    "ProfitOrLossAttributableToNonControllingInterests": 10000,  # Changed from 'ProfitLossAttributableToNoncontrollingInterests'
    "RepairsAndMaintenanceExpense": 15000,
    "SalesAndMarketingExpense": 25000,
    "OtherExpenses": 10000,  # Changed from 'OtherExpensesByNature'
    "OtherGainsOrLosses": 0,  # Changed from 'OtherGainsLosses'
    "FinanceCosts": 8000,
    "ShareOfProfitOrLossOfAssociatesAndJointVentures": 0,  # Changed from 'ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod'
    "ProfitOrLossBeforeTax": 150000,  # Changed from 'ProfitLossBeforeTaxation'
    "TaxExpenseOrBenefit": 30000,  # Changed from 'TaxExpenseBenefitContinuingOperations'
  },
  "notes": {
    "tradeAndOtherReceivables": {
      "ReceivablesFromThirdParties": 25000,  # Changed from 'TradeAndOtherReceivablesDueFromThirdParties'
      "ReceivablesFromRelatedParties": 15000,  # Changed from 'TradeAndOtherReceivablesDueFromRelatedParties'
      "UnbilledReceivables": 5000,
      "OtherReceivables": 2000,
      "TotalTradeAndOtherReceivables": 45000  # Changed from 'TradeAndOtherReceivables'
    },
    "tradeAndOtherPayables": {
      "PayablesToThirdParties": 20000,  # Changed from 'TradeAndOtherPayablesDueToThirdParties'
      "PayablesToRelatedParties": 10000,  # Changed from 'TradeAndOtherPayablesDueToRelatedParties'
      "DeferredIncome": 3000,
      "OtherPayables": 2000,
      "TotalTradeAndOtherPayables": 55000  # Changed from 'TradeAndOtherPayables'
    },
    "revenue": {
      "RevenueFromPropertyAtPointInTime": 0,  # Changed from 'RevenueFromPropertyTransferredAtPointInTime'
      "RevenueFromGoodsAtPointInTime": 0,  # Changed from 'RevenueFromGoodsTransferredAtPointInTime'
      "RevenueFromServicesAtPointInTime": 0,  # Changed from 'RevenueFromServicesTransferredAtPointInTime'
      "RevenueFromPropertyOverTime": 5000,  # Changed from 'RevenueFromPropertyTransferredOverTime'
      "RevenueFromConstructionContracts": 3000,  # Changed from 'RevenueFromConstructionContractsOverTime'
      "RevenueFromServicesOverTime": 7000,  # Changed from 'RevenueFromServicesTransferredOverTime'
      "OtherRevenue": 1000,
      "TotalRevenue": 120000  # Changed from 'Revenue'
    }
  }
}

 


### USAGE EXAMPLE

In [ ]:
# Sample test run


# OPTIONS TO CHOOSE FROM
# dummy_profit_loss_data
# dummy_data
# dummy_data_noise

# CHANGE VALUE HERE FOR TESTING
dummy_data= dummy_data_noise


dummy_data_json = json.dumps(dummy_data, indent=4)
result_mapping = await financial_statement_agent.run(
    f'Please map this financial statement data: {dummy_data_json}',
    deps=financial_deps
)

In [ ]:
# Print Test Run
debug(result_mapping)
print("\nMapped Statement:")
pprint.pprint(result_mapping.data)

# TAGGING IMPLEMENTATION

In [ ]:
# Imports and environment variables
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
import json



# Get API keys from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "")

### MODELS

In [ ]:
# Tagging Models
class FinancialTag(BaseModel):
    """Model for a single financial tag with standard XBRL attributes from Singapore ACRA taxonomy"""
    prefix: str = Field("sg-as", description="Taxonomy prefix (e.g., 'sg-as' for Singapore Account Standards)")
    element_name: str = Field(..., description="Official element name in the taxonomy (e.g., 'Revenue')")
    element_id: str = Field(..., description="Full element ID including prefix (e.g., 'sg-as_Revenue')")
    abstract: bool = Field(default=False, description="Whether tag represents an abstract concept")
    data_type: str = Field("xbrli:monetaryItemType", description="XBRL data type")
    balance_type: Optional[str] = Field(None, description="Balance type: 'debit' or 'credit'")
    period_type: str = Field("instant", description="Time period type: 'instant' or 'duration'")
    substitution_group: str = Field("xbrli:item", description="XBRL substitution group")

class TaggedValue(BaseModel):
    """A financial value with associated XBRL tags"""
    value: float = Field(..., description="The financial value")
    tags: List[FinancialTag] = Field(default_factory=list, description="Tags associated with this element name")
    
    def add_tag(self, tag: FinancialTag) -> None:
        """Add a tag to this value"""
        self.tags.append(tag)
        
    def __str__(self):
        """String representation of the value"""
        return f"{self.value}"

#----------------------------------------------------------------------------------------------------------------------------------------------
# FILLING INFORMATION
#----------------------------------------------------------------------------------------------------------------------------------------------

class FilingInformationWithTags(BaseModel):
    """Filing information with tags"""
    NameOfCompany: TaggedValue
    UniqueEntityNumber: TaggedValue
    CurrentPeriodStartDate: TaggedValue
    CurrentPeriodEndDate: TaggedValue
    PriorPeriodStartDate: Optional[TaggedValue] = None
    TypeOfXBRLFiling: TaggedValue
    NatureOfFinancialStatementsCompanyLevelOrConsolidated: TaggedValue
    TypeOfAccountingStandardUsedToPrepareFinancialStatements: TaggedValue
    DateOfAuthorisationForIssueOfFinancialStatements: TaggedValue
    TypeOfStatementOfFinancialPosition: TaggedValue
    WhetherTheFinancialStatementsArePreparedOnGoingConcernBasis: TaggedValue
    WhetherThereAreAnyChangesToComparativeAmounts: Optional[TaggedValue] = None
    DescriptionOfPresentationCurrency: TaggedValue
    DescriptionOfFunctionalCurrency: TaggedValue
    LevelOfRoundingUsedInFinancialStatements: TaggedValue
    DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities: TaggedValue
    PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice: TaggedValue
    WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees: TaggedValue
    NameOfParentEntity: Optional[TaggedValue] = None
    NameOfUltimateParentOfGroup: Optional[TaggedValue] = None
    TaxonomyVersion: TaggedValue
    NameAndVersionOfSoftwareUsedToGenerateXBRLFile: TaggedValue
    HowWasXBRLFilePrepared: TaggedValue
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire filing information")

class DirectorsStatementWithTags(BaseModel):
    """Directors' statement with tags"""
    WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView: TaggedValue
    WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement: TaggedValue
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire directors' statement")

class AuditReportWithTags(BaseModel):
    """Audit report with tags"""
    TypeOfAuditOpinionInIndependentAuditorsReport: TaggedValue
    AuditingStandardsUsedToConductTheAudit: Optional[TaggedValue] = None
    WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern: Optional[TaggedValue] = None
    WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKept: Optional[TaggedValue] = None
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire audit report")

#----------------------------------------------------------------------------------------------------------------------------------------------
# STATEMENT OF FINANCIAL POSITION
#----------------------------------------------------------------------------------------------------------------------------------------------

# Current Assets with Tags
class CurrentAssetsWithTags(BaseModel):
    """Current assets section with tags"""
    CashAndBankBalances: Optional[TaggedValue] = None
    TradeAndOtherReceivablesCurrent: Optional[TaggedValue] = None
    CurrentFinanceLeaseReceivables: Optional[TaggedValue] = None
    CurrentDerivativeFinancialAssets: Optional[TaggedValue] = None
    CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss: Optional[TaggedValue] = None
    OtherCurrentFinancialAssets: Optional[TaggedValue] = None
    DevelopmentProperties: Optional[TaggedValue] = None
    Inventories: Optional[TaggedValue] = None
    OtherCurrentNonfinancialAssets: Optional[TaggedValue] = None
    NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners: Optional[TaggedValue] = None
    CurrentAssets: TaggedValue
    
# Non-Current Assets with Tags
class NonCurrentAssetsWithTags(BaseModel):
    """Non-current assets section with tags"""
    TradeAndOtherReceivablesNoncurrent: Optional[TaggedValue] = None
    NoncurrentFinanceLeaseReceivables: Optional[TaggedValue] = None
    NoncurrentDerivativeFinancialAssets: Optional[TaggedValue] = None
    NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss: Optional[TaggedValue] = None
    OtherNoncurrentFinancialAssets: Optional[TaggedValue] = None
    PropertyPlantAndEquipment: Optional[TaggedValue] = None
    InvestmentProperties: Optional[TaggedValue] = None
    Goodwill: Optional[TaggedValue] = None
    IntangibleAssetsOtherThanGoodwill: Optional[TaggedValue] = None
    InvestmentsInSubsidiariesAssociatesOrJointVentures: Optional[TaggedValue] = None
    DeferredTaxAssets: Optional[TaggedValue] = None
    OtherNoncurrentNonfinancialAssets: Optional[TaggedValue] = None
    NoncurrentAssets: TaggedValue

# Current Liabilities with Tags
class CurrentLiabilitiesWithTags(BaseModel):
    """Current liabilities section with tags"""
    TradeAndOtherPayablesCurrent: Optional[TaggedValue] = None
    CurrentLoansAndBorrowings: Optional[TaggedValue] = None
    CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss: Optional[TaggedValue] = None
    CurrentFinanceLeaseLiabilities: Optional[TaggedValue] = None
    OtherCurrentFinancialLiabilities: Optional[TaggedValue] = None
    CurrentIncomeTaxLiabilities: Optional[TaggedValue] = None
    CurrentProvisions: Optional[TaggedValue] = None
    OtherCurrentNonfinancialLiabilities: Optional[TaggedValue] = None
    LiabilitiesClassifiedAsHeldForSale: Optional[TaggedValue] = None
    CurrentLiabilities: TaggedValue

# Non-Current Liabilities with Tags
class NonCurrentLiabilitiesWithTags(BaseModel):
    """Non-current liabilities section with tags"""
    TradeAndOtherPayablesNoncurrent: Optional[TaggedValue] = None
    NoncurrentLoansAndBorrowings: Optional[TaggedValue] = None
    NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss: Optional[TaggedValue] = None
    NoncurrentFinanceLeaseLiabilities: Optional[TaggedValue] = None
    OtherNoncurrentFinancialLiabilities: Optional[TaggedValue] = None
    DeferredTaxLiabilities: Optional[TaggedValue] = None
    NoncurrentProvisions: Optional[TaggedValue] = None
    OtherNoncurrentNonfinancialLiabilities: Optional[TaggedValue] = None
    NoncurrentLiabilities: TaggedValue

# Equity with Tags
class EquityWithTags(BaseModel):
    """Equity section with tags"""
    ShareCapital: TaggedValue
    TreasuryShares: Optional[TaggedValue] = None
    AccumulatedProfitsLosses: TaggedValue
    ReservesOtherThanAccumulatedProfitsLosses: Optional[TaggedValue] = None
    NoncontrollingInterests: Optional[TaggedValue] = None
    Equity: TaggedValue

# Statement of Financial Position with Tags
class StatementOfFinancialPositionWithTags(BaseModel):
    """Statement of financial position with tags"""
    currentAssets: CurrentAssetsWithTags
    nonCurrentAssets: NonCurrentAssetsWithTags
    Assets: TaggedValue
    currentLiabilities: CurrentLiabilitiesWithTags
    nonCurrentLiabilities: NonCurrentLiabilitiesWithTags  
    Liabilities: TaggedValue
    equity: EquityWithTags
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire statement")
    
    class Config:
        extra = "forbid"
        
    def validate_balance(self) -> bool:
        """
        Validate that Assets = Liabilities + Equity
        
        Returns:
            bool: True if balanced, False otherwise
        """
        # Allow for a small rounding difference
        tolerance = 0.01
        
        # Get values for comparison
        assets = self.Assets.value
        liabilities = self.Liabilities.value
        equity = self.equity.Equity.value
        
        balance_diff = abs(assets - (liabilities + equity))
        return balance_diff <= tolerance
    
    def add_meta_tag(self, tag: FinancialTag) -> None:
        """Add a statement-level tag"""
        self.meta_tags.append(tag)
    
    def get_all_tags(self) -> Dict[str, List[FinancialTag]]:
        """Get all tags organized by field path"""
        all_tags = {}
        
        # Add meta tags
        if self.meta_tags:
            all_tags["statement"] = self.meta_tags
        
        # Helper function to extract tags from a section
        def extract_section_tags(section, prefix):
            for field_name, field_value in section.__dict__.items():
                if isinstance(field_value, TaggedValue) and field_value.tags:
                    all_tags[f"{prefix}.{field_name}"] = field_value.tags
        
        # Extract tags from each section
        extract_section_tags(self.currentAssets, "currentAssets")
        extract_section_tags(self.nonCurrentAssets, "nonCurrentAssets")
        extract_section_tags(self.currentLiabilities, "currentLiabilities")
        extract_section_tags(self.nonCurrentLiabilities, "nonCurrentLiabilities")
        extract_section_tags(self.equity, "equity")
        
        # Add top-level fields
        if self.Assets.tags:
            all_tags["Assets"] = self.Assets.tags
        if self.Liabilities.tags:
            all_tags["Liabilities"] = self.Liabilities.tags
            
        return all_tags

#----------------------------------------------------------------------------------------------------------------------------------------------
# STATEMENT OF PROFIT OR LOSS
#----------------------------------------------------------------------------------------------------------------------------------------------

class StatementOfProfitOrLossWithTags(BaseModel):
    """Statement of Profit or Loss that supports tags on each field"""
    revenue: TaggedValue
    other_income: Optional[TaggedValue] = None
    employee_benefits_expense: Optional[TaggedValue] = None
    depreciation_expense: Optional[TaggedValue] = None
    amortisation_expense: Optional[TaggedValue] = None
    repairs_maintenance_expense: Optional[TaggedValue] = None
    sales_marketing_expense: Optional[TaggedValue] = None
    other_expenses: Optional[TaggedValue] = None
    other_gains_losses: Optional[TaggedValue] = None
    finance_costs_net: Optional[TaggedValue] = None
    share_of_profit_loss_associates: Optional[TaggedValue] = None
    profit_loss_before_taxation: TaggedValue
    income_tax_expense_benefit: TaggedValue
    profit_loss_discontinued_operations: Optional[TaggedValue] = None
    total_profit_loss: Optional[TaggedValue] = None
    profit_loss_attributable_to_owners: Optional[TaggedValue] = None
    profit_loss_attributable_to_non_controlling: Optional[TaggedValue] = None
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire statement")

#----------------------------------------------------------------------------------------------------------------------------------------------
# NOTES SECTION
#----------------------------------------------------------------------------------------------------------------------------------------------

# Notes section with tags
class TradeAndOtherReceivablesWithTags(BaseModel):
    """Trade and other receivables with tags"""
    TradeAndOtherReceivablesDueFromThirdParties: Optional[TaggedValue] = None
    TradeAndOtherReceivablesDueFromRelatedParties: Optional[TaggedValue] = None
    UnbilledReceivables: Optional[TaggedValue] = None
    OtherReceivables: Optional[TaggedValue] = None
    TradeAndOtherReceivables: TaggedValue
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for trade and other receivables section")

class TradeAndOtherPayablesWithTags(BaseModel):
    """Trade and other payables with tags"""
    TradeAndOtherPayablesDueToThirdParties: Optional[TaggedValue] = None
    TradeAndOtherPayablesDueToRelatedParties: Optional[TaggedValue] = None
    DeferredIncome: Optional[TaggedValue] = None
    OtherPayables: Optional[TaggedValue] = None
    TradeAndOtherPayables: TaggedValue
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for trade and other payables section")

class RevenueWithTags(BaseModel):
    """Revenue details with tags"""
    RevenueFromPropertyTransferredAtPointInTime: Optional[TaggedValue] = None
    RevenueFromGoodsTransferredAtPointInTime: Optional[TaggedValue] = None
    RevenueFromServicesTransferredAtPointInTime: Optional[TaggedValue] = None
    RevenueFromPropertyTransferredOverTime: Optional[TaggedValue] = None
    RevenueFromConstructionContractsOverTime: Optional[TaggedValue] = None
    RevenueFromServicesTransferredOverTime: Optional[TaggedValue] = None
    OtherRevenue: Optional[TaggedValue] = None
    Revenue: TaggedValue
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for revenue section")

class NotesWithTags(BaseModel):
    """Notes to financial statements with tags"""
    tradeAndOtherReceivables: TradeAndOtherReceivablesWithTags
    tradeAndOtherPayables: TradeAndOtherPayablesWithTags
    revenue: RevenueWithTags
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire notes section")
    
    class Config:
        title = "Comprehensive financial statement schema with XBRL tags compliant with Singapore Simplified XBRL requirements"

#----------------------------------------------------------------------------------------------------------------------------------------------
# COMBINED MODEL
#----------------------------------------------------------------------------------------------------------------------------------------------

class PartialXBRLWithTags(BaseModel):
    """Singapore XBRL schema with tags"""
    filingInformation: FilingInformationWithTags
    directorsStatement: DirectorsStatementWithTags
    auditReport: AuditReportWithTags
    statementOfFinancialPosition: StatementOfFinancialPositionWithTags
    incomeStatement: StatementOfProfitOrLossWithTags  # Using the existing class from your code
    notes: NotesWithTags
    meta_tags: List[FinancialTag] = Field(default_factory=list, description="Tags for the entire XBRL document")
    
    def get_all_tags(self) -> Dict[str, List[FinancialTag]]:
        """Get all tags organized by field path"""
        all_tags = {}
        
        # Add document-level meta tags
        if self.meta_tags:
            all_tags["document"] = self.meta_tags
            
        # Add section-level meta tags
        if self.filingInformation.meta_tags:
            all_tags["filingInformation"] = self.filingInformation.meta_tags
        if self.directorsStatement.meta_tags:
            all_tags["directorsStatement"] = self.directorsStatement.meta_tags
        if self.auditReport.meta_tags:
            all_tags["auditReport"] = self.auditReport.meta_tags
        if self.statementOfFinancialPosition.meta_tags:
            all_tags["statementOfFinancialPosition"] = self.statementOfFinancialPosition.meta_tags
        if self.incomeStatement.meta_tags:
            all_tags["incomeStatement"] = self.incomeStatement.meta_tags
        if self.notes.meta_tags:
            all_tags["notes"] = self.notes.meta_tags
            
        # Add field-level tags from each section using a recursive helper function
        def extract_tagged_values(obj, prefix=""):
            for field_name, field_value in obj.__dict__.items():
                if field_name == "meta_tags":
                    continue
                    
                field_path = f"{prefix}.{field_name}" if prefix else field_name
                
                if isinstance(field_value, TaggedValue) and field_value.tags:
                    all_tags[field_path] = field_value.tags
                elif isinstance(field_value, BaseModel):
                    extract_tagged_values(field_value, field_path)
        
        # Extract tags from each section
        extract_tagged_values(self.filingInformation, "filingInformation")
        extract_tagged_values(self.directorsStatement, "directorsStatement")
        extract_tagged_values(self.auditReport, "auditReport")
        extract_tagged_values(self.statementOfFinancialPosition, "statementOfFinancialPosition")
        extract_tagged_values(self.incomeStatement, "incomeStatement") 
        extract_tagged_values(self.notes, "notes")
        
        return all_tags

### DEPENDENCIES

In [ ]:
# Tagging Dependencies
@dataclass
class XBRLTaxonomyDependencies:
    """Dependencies for XBRL tagging operations"""
    taxonomy_name: str
    entity_name: str
    mandatory_fields: Dict[str, bool]
    field_tags: Dict[str, List[FinancialTag]]
    statement_tags: List[FinancialTag]
    reporting_year: Optional[str] = None # Set to None since we're not using it currently

#### TAXONOMY TAGS DEPENDENCIES

In [ ]:
# Filing information XBRL taxonomy tags

SG_XBRL_FILING_TAGS = {
    "DisclosureOfFilingInformationAbstract": [
        FinancialTag(
            prefix="sg-dei",
            element_name="DisclosureOfFilingInformationAbstract",
            element_id="sg-dei_DisclosureOfFilingInformationAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element for filing information section that groups related disclosure items"
        )
    ],
    "NameOfCompany": [
        FinancialTag(
            prefix="sg-dei",
            element_name="NameOfCompany",
            element_id="sg-dei_NameOfCompany",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The legal registered name of the entity filing the financial statements as recorded with ACRA"
        )
    ],
    "UniqueEntityNumber": [
        FinancialTag(
            prefix="sg-dei",
            element_name="UniqueEntityNumber",
            element_id="sg-dei_UniqueEntityNumber",
            abstract=False,
            data_type="sg-types:UENItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The Unique Entity Number (UEN) issued by ACRA, formatted as 8 digits followed by 1 uppercase letter"
        )
    ],
    "CurrentPeriodStartDate": [
        FinancialTag(
            prefix="sg-dei",
            element_name="CurrentPeriodStartDate",
            element_id="sg-dei_CurrentPeriodStartDate",
            abstract=False,
            data_type="xbrli:dateItemType",
            balance_type=None,
            period_type="instant",
            substitution_group="xbrli:item",
            description="The first day of the current financial reporting period (e.g., January 1, 2023)"
        )
    ],
    "CurrentPeriodEndDate": [
        FinancialTag(
            prefix="sg-dei",
            element_name="CurrentPeriodEndDate",
            element_id="sg-dei_CurrentPeriodEndDate",
            abstract=False,
            data_type="xbrli:dateItemType",
            balance_type=None,
            period_type="instant",
            substitution_group="xbrli:item",
            description="The last day of the current financial reporting period (e.g., December 31, 2023)"
        )
    ],
    "PriorPeriodStartDate": [
        FinancialTag(
            prefix="sg-dei",
            element_name="PriorPeriodStartDate",
            element_id="sg-dei_PriorPeriodStartDate",
            abstract=False,
            data_type="xbrli:dateItemType",
            balance_type=None,
            period_type="instant",
            substitution_group="xbrli:item",
            description="The first day of the previous financial reporting period used for comparative figures"
        )
    ],
    "TypeOfXBRLFiling": [
        FinancialTag(
            prefix="sg-dei",
            element_name="TypeOfXBRLFiling",
            element_id="sg-dei_TypeOfXBRLFiling",
            abstract=False,
            data_type="sg-types:XBRLFilingItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Indicates whether the filing contains full or partial XBRL information (Full/Partial)"
        )
    ],
    "NatureOfFinancialStatementsCompanyLevelOrConsolidated": [
        FinancialTag(
            prefix="sg-dei",
            element_name="NatureOfFinancialStatementsCompanyLevelOrConsolidated",
            element_id="sg-dei_NatureOfFinancialStatementsCompanyLevelOrConsolidated",
            abstract=False,
            data_type="sg-types:NatureOfFinancialStatementsItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Specifies whether the statements are for the company alone (separate) or for the consolidated group"
        )
    ],
    "TypeOfAccountingStandardUsedToPrepareFinancialStatements": [
        FinancialTag(
            prefix="sg-dei",
            element_name="TypeOfAccountingStandardUsedToPrepareFinancialStatements",
            element_id="sg-dei_TypeOfAccountingStandardUsedToPrepareFinancialStatements",
            abstract=False,
            data_type="sg-types:AccountingStandardsItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The accounting standards framework used to prepare the financial statements (SFRS, SFRS for SE, IFRS, or Other)"
        )
    ],
    "DateOfAuthorisationForIssueOfFinancialStatements": [
        FinancialTag(
            prefix="sg-as",
            element_name="DateOfAuthorisationForIssueOfFinancialStatements",
            element_id="sg-as_DateOfAuthorisationForIssueOfFinancialStatements",
            abstract=False,
            data_type="xbrli:dateItemType",
            balance_type=None,
            period_type="instant",
            substitution_group="xbrli:item",
            description="Date when the directors authorized the financial statements for issuance, typically the board approval date"
        )
    ],
    "TypeOfStatementOfFinancialPosition": [
        FinancialTag(
            prefix="sg-dei",
            element_name="TypeOfStatementOfFinancialPosition",
            element_id="sg-dei_TypeOfStatementOfFinancialPosition",
            abstract=False,
            data_type="sg-types:StatementOfFinancialPositionItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Format used for the balance sheet - either classified (current/non-current) or liquidity-based"
        )
    ],
    "WhetherFinancialStatementsArePreparedOnGoingConcernBasis": [
        FinancialTag(
            prefix="sg-dei",
            element_name="WhetherFinancialStatementsArePreparedOnGoingConcernBasis",
            element_id="sg-dei_WhetherFinancialStatementsArePreparedOnGoingConcernBasis",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Indicates whether the entity is considered a going concern for the preparation of financial statements"
        )
    ],
    "WhetherThereAreChangesToComparativeAmountsDueToRestatementsReclassificationOrOtherReasons": [
        FinancialTag(
            prefix="sg-dei",
            element_name="WhetherThereAreChangesToComparativeAmountsDueToRestatementsReclassificationOrOtherReasons",
            element_id="sg-dei_WhetherThereAreChangesToComparativeAmountsDueToRestatementsReclassificationOrOtherReasons",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Indicates if prior year figures have been restated, reclassified, or otherwise modified"
        )
    ],
    "DescriptionOfPresentationCurrency": [
        FinancialTag(
            prefix="sg-dei",
            element_name="DescriptionOfPresentationCurrency",
            element_id="sg-dei_DescriptionOfPresentationCurrency",
            abstract=False,
            data_type="sg-types:CurrencyCodeItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The ISO 4217 currency code used for presenting the financial statements (e.g., SGD, USD)"
        )
    ],
    "DescriptionOfFunctionalCurrency": [
        FinancialTag(
            prefix="sg-as",
            element_name="DescriptionOfFunctionalCurrency",
            element_id="sg-as_DescriptionOfFunctionalCurrency",
            abstract=False,
            data_type="sg-types:CurrencyCodeItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The ISO 4217 currency code of the primary economic environment in which the entity operates"
        )
    ],
    "LevelOfRoundingUsedInFinancialStatements": [
        FinancialTag(
            prefix="sg-dei",
            element_name="LevelOfRoundingUsedInFinancialStatements",
            element_id="sg-dei_LevelOfRoundingUsedInFinancialStatements",
            abstract=False,
            data_type="sg-types:LevelOfRoundingItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The level of rounding applied to numeric values (Units, Thousands, Millions)"
        )
    ],
    "DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities": [
        FinancialTag(
            prefix="sg-as",
            element_name="DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities",
            element_id="sg-as_DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="A brief description of the company's main business activities and operations"
        )
    ],
    "PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice": [
        FinancialTag(
            prefix="sg-as",
            element_name="PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice",
            element_id="sg-as_PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The main location where business is conducted if different from the registered office address"
        )
    ],
    "WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees": [
        FinancialTag(
            prefix="sg-dei",
            element_name="WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees",
            element_id="sg-dei_WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Indicates whether the company or group has more than 50 employees, affecting certain disclosure requirements"
        )
    ],
    "NameOfParentEntity": [
        FinancialTag(
            prefix="sg-as",
            element_name="NameOfParentEntity",
            element_id="sg-as_NameOfParentEntity",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The name of the immediate parent company that owns or controls the reporting entity"
        )
    ],
    "NameOfUltimateParentOfGroup": [
        FinancialTag(
            prefix="sg-as",
            element_name="NameOfUltimateParentOfGroup",
            element_id="sg-as_NameOfUltimateParentOfGroup",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The name of the topmost parent company in the corporate structure that ultimately controls the reporting entity"
        )
    ],
    "DetailsOfInstanceDocumentAbstract": [
        FinancialTag(
            prefix="sg-dei",
            element_name="DetailsOfInstanceDocumentAbstract",
            element_id="sg-dei_DetailsOfInstanceDocumentAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element that groups technical information about the XBRL instance document"
        )
    ],
    "TaxonomyVersion": [
        FinancialTag(
            prefix="sg-dei",
            element_name="TaxonomyVersion",
            element_id="sg-dei_TaxonomyVersion",
            abstract=False,
            data_type="sg-types:TaxonomyVersionItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The version of the ACRA XBRL taxonomy used for preparing the filing (e.g., 2022.2)"
        )
    ],
    "NameAndVersionOfSoftwareUsedToGenerateInstanceDocument": [
        FinancialTag(
            prefix="sg-dei",
            element_name="NameAndVersionOfSoftwareUsedToGenerateInstanceDocument",
            element_id="sg-dei_NameAndVersionOfSoftwareUsedToGenerateInstanceDocument",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Information about the software application and version used to create the XBRL filing"
        )
    ],
    "HowWasXBRLInstanceDocumentPrepared": [
        FinancialTag(
            prefix="sg-dei",
            element_name="HowWasXBRLInstanceDocumentPrepared",
            element_id="sg-dei_HowWasXBRLInstanceDocumentPrepared",
            abstract=False,
            data_type="sg-types:PreparationOfXBRLFileItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The method used to create the XBRL filing (Automated, Manual, or Hybrid)"
        )
    ]
}
# statement-level tags
SG_XBRL_FILING_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-dei",
        element_name="DisclosureOfFilingInformationAbstract",
        element_id="sg-dei_DisclosureOfFilingInformationAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Abstract container element that represents the entire filing information section of the report"
    ),
    FinancialTag(
        prefix="sg-dei",
        element_name="DetailsOfInstanceDocumentAbstract",
        element_id="sg-dei_DetailsOfInstanceDocumentAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Abstract container element that groups technical metadata about the XBRL document itself"
    )
]

MANDATORY_FILING_TAGS = {
    "NameOfCompany": True,
    "UniqueEntityNumber": True,
    "CurrentPeriodStartDate": True,
    "CurrentPeriodEndDate": True,
    "TypeOfXBRLFiling": True,
    "NatureOfFinancialStatementsCompanyLevelOrConsolidated": True,
    "TypeOfAccountingStandardUsedToPrepareFinancialStatements": True,
    "DateOfAuthorisationForIssueOfFinancialStatements": True,
    "TypeOfStatementOfFinancialPosition": True,
    "WhetherFinancialStatementsArePreparedOnGoingConcernBasis": True,
    "DescriptionOfPresentationCurrency": True,
    "DescriptionOfFunctionalCurrency": True,
    "LevelOfRoundingUsedInFinancialStatements": True,
    "DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities": True,
    "PrincipalPlaceOfBusinessIfDifferentFromRegisteredOffice": True,
    "WhetherCompanyOrGroupIfConsolidatedAccountsArePreparedHasMoreThan50Employees": True,
    "TaxonomyVersion": True
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Statement by directors XBRL taxonomy tags
#----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_DIRECTORS_TAGS = {
    "DisclosureInStatementByDirectorsAbstract": [
        FinancialTag(
            prefix="sg-dei",
            element_name="DisclosureInStatementByDirectorsAbstract",
            element_id="sg-dei_DisclosureInStatementByDirectorsAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element that groups all directors' statement disclosures"
        )
    ],
    "WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView": [
        FinancialTag(
            prefix="sg-dei",
            element_name="WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView",
            element_id="sg-dei_WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Directors' confirmation that financial statements give a true and fair view of the company's financial position and performance"
        )
    ],
    "WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement": [
        FinancialTag(
            prefix="sg-dei",
            element_name="WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement",
            element_id="sg-dei_WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Directors' assessment of the company's solvency and ability to pay debts when due (going concern statement)"
        )
    ]
}

# statement-level tags
SG_XBRL_DIRECTORS_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-dei",
        element_name="DisclosureInStatementByDirectorsAbstract",
        element_id="sg-dei_DisclosureInStatementByDirectorsAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container for the entire statement by directors section"
    )
]

MANDATORY_DIRECTORS_TAGS = {
    "WhetherInDirectorsOpinionFinancialStatementsAreDrawnUpSoAsToExhibitATrueAndFairView": True,
    "WhetherThereAreReasonableGroundsToBelieveThatCompanyWillBeAbleToPayItsDebtsAsAndWhenTheyFallDueAtDateOfStatement": True
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Independent auditors' report XBRL taxonomy tags
#----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_AUDIT_TAGS = {
    "DisclosuresInIndependentAuditorsReportAbstract": [
        FinancialTag(
            prefix="sg-ssa",
            element_name="DisclosuresInIndependentAuditorsReportAbstract",
            element_id="sg-ssa_DisclosuresInIndependentAuditorsReportAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element that groups all disclosures related to the independent auditors' report"
        )
    ],
    "TypeOfAuditOpinionInIndependentAuditorsReport": [
        FinancialTag(
            prefix="sg-ssa",
            element_name="TypeOfAuditOpinionInIndependentAuditorsReport",
            element_id="sg-ssa_TypeOfAuditOpinionInIndependentAuditorsReport",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The type of opinion expressed by the auditor in their report (Unqualified, Qualified, Adverse, or Disclaimer of Opinion)"
        )
    ],
    "AuditingStandardsUsedToConductAudit": [
        FinancialTag(
            prefix="sg-ssa",
            element_name="AuditingStandardsUsedToConductAudit",
            element_id="sg-ssa_AuditingStandardsUsedToConductAudit",
            abstract=False,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The framework of auditing standards used to conduct the audit (e.g., Singapore Standards on Auditing, International Standards on Auditing)"
        )
    ],
    "WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern": [
        FinancialTag(
            prefix="sg-ssa",
            element_name="WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern",
            element_id="sg-ssa_WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Indicates whether the auditor has identified any material uncertainty related to events or conditions that may cast significant doubt on the entity's ability to continue as a going concern"
        )
    ],
    "WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKeptInAccordanceWithCompaniesAct": [
        FinancialTag(
            prefix="sg-ssa",
            element_name="WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKeptInAccordanceWithCompaniesAct",
            element_id="sg-ssa_WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKeptInAccordanceWithCompaniesAct",
            abstract=False,
            data_type="sg-types:YesNoItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="The auditor's opinion on whether proper accounting and other records have been kept by the company as required by the Singapore Companies Act"
        )
    ]
}

# statement-level tags
SG_XBRL_AUDIT_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-ssa",
        element_name="DisclosuresInIndependentAuditorsReportAbstract",
        element_id="sg-ssa_DisclosuresInIndependentAuditorsReportAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container element for the entire independent auditors' report section"
    )
]

MANDATORY_AUDIT_TAGS = {
    "TypeOfAuditOpinionInIndependentAuditorsReport": True,
    "AuditingStandardsUsedToConductAudit": False,  # Optional based on Tag.md
    "WhetherThereIsAnyMaterialUncertaintyRelatingToGoingConcern": False,  # Optional based on Tag.md
    "WhetherInAuditorsOpinionAccountingAndOtherRecordsRequiredAreProperlyKeptInAccordanceWithCompaniesAct": False  # Optional based on Tag.md
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Statement of financial position XBRL taxonomy tags - Assets section
#----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_FINANCIAL_POSITION_TAGS = {
    "StatementOfFinancialPositionLineItems": [
        FinancialTag(
            prefix="sg-as",
            element_name="StatementOfFinancialPositionLineItems",
            element_id="sg-as_StatementOfFinancialPositionLineItems",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all statement of financial position line items"
        )
    ],
    "AssetsAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="AssetsAbstract",
            element_id="sg-as_AssetsAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all asset categories (current and non-current)"
        )
    ],
    "CurrentAssetsAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentAssetsAbstract",
            element_id="sg-as_CurrentAssetsAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for assets expected to be converted into cash or consumed within one year"
        )
    ],
    "CashAndBankBalances": [
        FinancialTag(
            prefix="sg-as",
            element_name="CashAndBankBalances",
            element_id="sg-as_CashAndBankBalances",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Cash on hand and demand deposits with banks, including short-term highly liquid investments"
        )
    ],
    "TradeAndOtherReceivablesCurrent": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivablesCurrent",
            element_id="sg-as_TradeAndOtherReceivablesCurrent",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts due from customers for goods sold or services rendered and other short-term receivables"
        )
    ],
    "CurrentFinanceLeaseReceivables": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentFinanceLeaseReceivables",
            element_id="sg-as_CurrentFinanceLeaseReceivables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Current portion of finance lease receivables where the entity is the lessor"
        )
    ],
    "CurrentDerivativeFinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentDerivativeFinancialAssets",
            element_id="sg-as_CurrentDerivativeFinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Current derivative instruments with positive fair values (e.g., forwards, swaps, options)"
        )
    ],
    "CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss",
            element_id="sg-as_CurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Current financial assets designated or classified as at fair value through profit or loss"
        )
    ],
    "OtherCurrentFinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherCurrentFinancialAssets",
            element_id="sg-as_OtherCurrentFinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other current financial assets not specifically classified elsewhere"
        )
    ],
    "DevelopmentProperties": [
        FinancialTag(
            prefix="sg-as",
            element_name="DevelopmentProperties",
            element_id="sg-as_DevelopmentProperties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Properties acquired or being constructed for sale in the ordinary course of business"
        )
    ],
    "Inventories": [
        FinancialTag(
            prefix="sg-as",
            element_name="Inventories",
            element_id="sg-as_Inventories",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Assets held for sale in ordinary course of business, in production, or as materials to be consumed"
        )
    ],
    "OtherCurrentNonfinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherCurrentNonfinancialAssets",
            element_id="sg-as_OtherCurrentNonfinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other current non-financial assets such as prepayments and advances"
        )
    ],
    "NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners",
            element_id="sg-as_NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSaleOrAsHeldForDistributionToOwners",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Non-current assets or disposal groups held for sale or distribution to owners"
        )
    ],
    "CurrentAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentAssets",
            element_id="sg-as_CurrentAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all current assets expected to be converted to cash or consumed within one operating cycle"
        )
    ],
    "NoncurrentAssetsAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentAssetsAbstract",
            element_id="sg-as_NoncurrentAssetsAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for assets not expected to be converted into cash within one year"
        )
    ],
    "TradeAndOtherReceivablesNoncurrent": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivablesNoncurrent",
            element_id="sg-as_TradeAndOtherReceivablesNoncurrent",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term receivables not expected to be collected within one year"
        )
    ],
    "NoncurrentFinanceLeaseReceivables": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentFinanceLeaseReceivables",
            element_id="sg-as_NoncurrentFinanceLeaseReceivables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term portion of finance lease receivables where the entity is the lessor"
        )
    ],
    "NoncurrentDerivativeFinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentDerivativeFinancialAssets",
            element_id="sg-as_NoncurrentDerivativeFinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term derivative instruments with positive fair values (e.g., interest rate swaps)"
        )
    ],
    "NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss",
            element_id="sg-as_NoncurrentFinancialAssetsMeasuredAtFairValueThroughProfitOrLoss",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term financial assets designated or classified as at fair value through profit or loss"
        )
    ],
    "OtherNoncurrentFinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherNoncurrentFinancialAssets",
            element_id="sg-as_OtherNoncurrentFinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other long-term financial assets not specifically classified elsewhere"
        )
    ],
    "PropertyPlantAndEquipment": [
        FinancialTag(
            prefix="sg-as",
            element_name="PropertyPlantAndEquipment",
            element_id="sg-as_PropertyPlantAndEquipment",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Tangible assets used in operations over more than one period (e.g., land, buildings, machinery)"
        )
    ],
    "InvestmentProperties": [
        FinancialTag(
            prefix="sg-as",
            element_name="InvestmentProperties",
            element_id="sg-as_InvestmentProperties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Properties held to earn rentals or for capital appreciation rather than for use in production"
        )
    ],
    "Goodwill": [
        FinancialTag(
            prefix="sg-as",
            element_name="Goodwill",
            element_id="sg-as_Goodwill",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="An asset representing future economic benefits arising from other assets acquired in a business combination"
        )
    ],
    "IntangibleAssetsOtherThanGoodwill": [
        FinancialTag(
            prefix="sg-as",
            element_name="IntangibleAssetsOtherThanGoodwill",
            element_id="sg-as_IntangibleAssetsOtherThanGoodwill",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Identifiable non-monetary assets without physical substance (e.g., patents, trademarks, software)"
        )
    ],
    "InvestmentsInSubsidiariesAssociatesOrJointVentures": [
        FinancialTag(
            prefix="sg-as",
            element_name="InvestmentsInSubsidiariesAssociatesOrJointVentures",
            element_id="sg-as_InvestmentsInSubsidiariesAssociatesOrJointVentures",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term investments in subsidiaries, associates, or joint ventures"
        )
    ],
    "DeferredTaxAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="DeferredTaxAssets",
            element_id="sg-as_DeferredTaxAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Income tax recoverable in future periods in respect of deductible temporary differences"
        )
    ],
    "OtherNoncurrentNonfinancialAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherNoncurrentNonfinancialAssets",
            element_id="sg-as_OtherNoncurrentNonfinancialAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other long-term non-financial assets not specifically classified elsewhere"
        )
    ],
    "NoncurrentAssets": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentAssets",
            element_id="sg-as_NoncurrentAssets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all non-current assets with economic benefits expected beyond one year"
        )
    ],
    "Assets": [
        FinancialTag(
            prefix="sg-as",
            element_name="Assets",
            element_id="sg-as_Assets",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all current and non-current resources controlled by the entity from which future economic benefits are expected"
        )
    ],
    "LiabilitiesAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="LiabilitiesAbstract",
            element_id="sg-as_LiabilitiesAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all liability categories (current and non-current)"
        )
    ],
    "CurrentLiabilitiesAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentLiabilitiesAbstract",
            element_id="sg-as_CurrentLiabilitiesAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all obligations expected to be settled within one operating cycle"
        )
    ],
    "TradeAndOtherPayablesCurrent": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayablesCurrent",
            element_id="sg-as_TradeAndOtherPayablesCurrent",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts owed to suppliers for goods and services received, and other short-term payables"
        )
    ],
    "CurrentLoansAndBorrowings": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentLoansAndBorrowings",
            element_id="sg-as_CurrentLoansAndBorrowings",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Short-term debt and current portions of long-term debt due within one year"
        )
    ],
    "CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss",
            element_id="sg-as_CurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Current financial liabilities designated or classified as at fair value through profit or loss"
        )
    ],
    "CurrentFinanceLeaseLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentFinanceLeaseLiabilities",
            element_id="sg-as_CurrentFinanceLeaseLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Current portion of obligations under finance leases due within one year"
        )
    ],
    "OtherCurrentFinancialLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherCurrentFinancialLiabilities",
            element_id="sg-as_OtherCurrentFinancialLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other current financial liabilities not specifically classified elsewhere"
        )
    ],
    "CurrentIncomeTaxLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentIncomeTaxLiabilities",
            element_id="sg-as_CurrentIncomeTaxLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Unpaid income taxes for the current and prior periods"
        )
    ],
    "CurrentProvisions": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentProvisions",
            element_id="sg-as_CurrentProvisions",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Provisions expected to be settled within one year (e.g., warranties, legal claims)"
        )
    ],
    "OtherCurrentNonfinancialLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherCurrentNonfinancialLiabilities",
            element_id="sg-as_OtherCurrentNonfinancialLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other current non-financial obligations (e.g., deferred revenue, customer advances)"
        )
    ],
    "LiabilitiesClassifiedAsHeldForSale": [
        FinancialTag(
            prefix="sg-as",
            element_name="LiabilitiesClassifiedAsHeldForSale",
            element_id="sg-as_LiabilitiesClassifiedAsHeldForSale",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Liabilities associated with assets classified as held for sale"
        )
    ],
    "CurrentLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="CurrentLiabilities",
            element_id="sg-as_CurrentLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all liabilities expected to be settled within one operating cycle"
        )
    ],
    "NoncurrentLiabilitiesAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentLiabilitiesAbstract",
            element_id="sg-as_NoncurrentLiabilitiesAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all obligations expected to be settled beyond one year"
        )
    ],
    "TradeAndOtherPayablesNoncurrent": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayablesNoncurrent",
            element_id="sg-as_TradeAndOtherPayablesNoncurrent",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term payables not expected to be settled within one year"
        )
    ],
    "NoncurrentLoansAndBorrowings": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentLoansAndBorrowings",
            element_id="sg-as_NoncurrentLoansAndBorrowings",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term debt and borrowings due beyond one year (e.g., bonds, term loans)"
        )
    ],
    "NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss",
            element_id="sg-as_NoncurrentFinancialLiabilitiesMeasuredAtFairValueThroughProfitOrLoss",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term financial liabilities designated or classified as at fair value through profit or loss"
        )
    ],
    "NoncurrentFinanceLeaseLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentFinanceLeaseLiabilities",
            element_id="sg-as_NoncurrentFinanceLeaseLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term portion of obligations under finance leases due beyond one year"
        )
    ],
    "OtherNoncurrentFinancialLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherNoncurrentFinancialLiabilities",
            element_id="sg-as_OtherNoncurrentFinancialLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other long-term financial liabilities not specifically classified elsewhere"
        )
    ],
    "DeferredTaxLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="DeferredTaxLiabilities",
            element_id="sg-as_DeferredTaxLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Future income tax obligations from taxable temporary differences"
        )
    ],
    "NoncurrentProvisions": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentProvisions",
            element_id="sg-as_NoncurrentProvisions",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Long-term provisions expected to be settled beyond one year (e.g., asset retirement obligations)"
        )
    ],
    "OtherNoncurrentNonfinancialLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherNoncurrentNonfinancialLiabilities",
            element_id="sg-as_OtherNoncurrentNonfinancialLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other long-term non-financial obligations (e.g., long-term deferred revenue)"
        )
    ],
    "NoncurrentLiabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncurrentLiabilities",
            element_id="sg-as_NoncurrentLiabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all liabilities expected to be settled beyond one year"
        )
    ],
    "Liabilities": [
        FinancialTag(
            prefix="sg-as",
            element_name="Liabilities",
            element_id="sg-as_Liabilities",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total of all current and non-current obligations that are present obligations arising from past events"
        )
    ],
    "EquityAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="EquityAbstract",
            element_id="sg-as_EquityAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all equity components representing residual interest in the assets after deducting liabilities"
        )
    ],
    "ShareCapital": [
        FinancialTag(
            prefix="sg-as",
            element_name="ShareCapital",
            element_id="sg-as_ShareCapital",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Nominal value of issued share capital, including share premium"
        )
    ],
    "TreasuryShares": [
        FinancialTag(
            prefix="sg-as",
            element_name="TreasuryShares",
            element_id="sg-as_TreasuryShares",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Own shares repurchased and held by the entity, shown as a deduction from equity"
        )
    ],
    "AccumulatedProfitsLosses": [
        FinancialTag(
            prefix="sg-as",
            element_name="AccumulatedProfitsLosses",
            element_id="sg-as_AccumulatedProfitsLosses",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Cumulative net profits or losses that have not been distributed to shareholders or transferred to reserves"
        )
    ],
    "ReservesOtherThanAccumulatedProfitsLosses": [
        FinancialTag(
            prefix="sg-as",
            element_name="ReservesOtherThanAccumulatedProfitsLosses",
            element_id="sg-as_ReservesOtherThanAccumulatedProfitsLosses",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Various reserves such as revaluation, translation, hedging, and other statutory or discretionary reserves"
        )
    ],
    "NoncontrollingInterests": [
        FinancialTag(
            prefix="sg-as",
            element_name="NoncontrollingInterests",
            element_id="sg-as_NoncontrollingInterests",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Equity in a subsidiary not attributable to the parent (minority interests)"
        )
    ],
    "Equity": [
        FinancialTag(
            prefix="sg-as",
            element_name="Equity",
            element_id="sg-as_Equity",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total residual interest in the assets after deducting all liabilities (net assets)"
        )
    ]
}

# statement-level tags
SG_XBRL_FINANCIAL_POSITION_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-as",
        element_name="StatementOfFinancialPositionLineItems",
        element_id="sg-as_StatementOfFinancialPositionLineItems",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container element for the entire statement of financial position"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="AssetsAbstract",
        element_id="sg-as_AssetsAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for all asset categories in the statement of financial position"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="LiabilitiesAbstract",
        element_id="sg-as_LiabilitiesAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for all liability categories in the statement of financial position"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="EquityAbstract",
        element_id="sg-as_EquityAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for all equity components in the statement of financial position"
    )
]

MANDATORY_FINANCIAL_POSITION_TAGS = {
    "CurrentAssets": True,
    "NoncurrentAssets": True,
    "Assets": True,
    "CurrentLiabilities": True,
    "NoncurrentLiabilities": True,
    "Liabilities": True,
    "ShareCapital": True,
    "AccumulatedProfitsLosses": True,
    "Equity": True
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Income statement XBRL taxonomy tags
# -----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_INCOME_STATEMENT_TAGS = {
    "StatementOfProfitOrLossLineItems": [
        FinancialTag(
            prefix="sg-as",
            element_name="StatementOfProfitOrLossLineItems",
            element_id="sg-as_StatementOfProfitOrLossLineItems",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for all statement of profit or loss line items"
        )
    ],
    "ProfitLossAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossAbstract",
            element_id="sg-as_ProfitLossAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for profit or loss section items"
        )
    ],
    "Revenue": [
        FinancialTag(
            prefix="sg-as",
            element_name="Revenue",
            element_id="sg-as_Revenue",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Income arising from the ordinary activities of the entity (sales, fees, interest, dividends, etc.)"
        )
    ],
    "OtherIncome": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherIncome",
            element_id="sg-as_OtherIncome",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Income from sources other than primary operations (rental, dividend income, government grants, etc.)"
        )
    ],
    "EmployeeBenefitsExpense": [
        FinancialTag(
            prefix="sg-as",
            element_name="EmployeeBenefitsExpense",
            element_id="sg-as_EmployeeBenefitsExpense",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="All forms of compensation paid to employees (salaries, wages, CPF contributions, bonuses, etc.)"
        )
    ],
    "DepreciationExpense": [
        FinancialTag(
            prefix="sg-as",
            element_name="DepreciationExpense",
            element_id="sg-as_DepreciationExpense",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Systematic allocation of the depreciable amount of tangible assets over their useful lives"
        )
    ],
    "AmortisationExpense": [
        FinancialTag(
            prefix="sg-as",
            element_name="AmortisationExpense",
            element_id="sg-as_AmortisationExpense",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Systematic allocation of the depreciable amount of intangible assets over their useful lives"
        )
    ],
    "RepairsAndMaintenanceExpense": [
        FinancialTag(
            prefix="sg-as",
            element_name="RepairsAndMaintenanceExpense",
            element_id="sg-as_RepairsAndMaintenanceExpense",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Costs incurred to maintain and repair assets to keep them in good working condition"
        )
    ],
    "SalesAndMarketingExpense": [
        FinancialTag(
            prefix="sg-as",
            element_name="SalesAndMarketingExpense",
            element_id="sg-as_SalesAndMarketingExpense",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Expenses related to promoting and selling products or services (advertising, marketing, etc.)"
        )
    ],
    "OtherExpensesByNature": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherExpensesByNature",
            element_id="sg-as_OtherExpensesByNature",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Other operating expenses not specifically classified elsewhere (utilities, rent, professional fees, etc.)"
        )
    ],
    "OtherGainsLosses": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherGainsLosses",
            element_id="sg-as_OtherGainsLosses",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Gains or losses from non-operating activities (foreign exchange, asset disposals, fair value changes)"
        )
    ],
    "FinanceCosts": [
        FinancialTag(
            prefix="sg-as",
            element_name="FinanceCosts",
            element_id="sg-as_FinanceCosts",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Interest and other costs incurred in connection with borrowing funds (interest expense, finance charges)"
        )
    ],
    "ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod": [
        FinancialTag(
            prefix="sg-as",
            element_name="ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod",
            element_id="sg-as_ShareOfProfitLossOfAssociatesAndJointVenturesAccountedForUsingEquityMethod",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Entity's share of the profit or loss of equity-accounted associates and joint ventures"
        )
    ],
    "ProfitLossBeforeTaxation": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossBeforeTaxation",
            element_id="sg-as_ProfitLossBeforeTaxation",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Profit or loss before deduction of income tax expense (revenues minus expenses before tax)"
        )
    ],
    "TaxExpenseBenefitContinuingOperations": [
        FinancialTag(
            prefix="sg-as",
            element_name="TaxExpenseBenefitContinuingOperations",
            element_id="sg-as_TaxExpenseBenefitContinuingOperations",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Income tax expense or benefit for continuing operations (current and deferred tax)"
        )
    ],
    "ProfitLossFromDiscontinuedOperations": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossFromDiscontinuedOperations",
            element_id="sg-as_ProfitLossFromDiscontinuedOperations",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Profit or loss from discontinued operations (business components disposed of or held for sale)"
        )
    ],
    "ProfitLoss": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLoss",
            element_id="sg-as_ProfitLoss",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Total profit or loss for the period (net income or net loss for the reporting period)"
        )
    ],
    "ProfitLossAttributableToAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossAttributableToAbstract",
            element_id="sg-as_ProfitLossAttributableToAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for allocation of profit or loss between parent owners and non-controlling interests"
        )
    ],
    "ProfitLossAttributableToOwnersOfCompany": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossAttributableToOwnersOfCompany",
            element_id="sg-as_ProfitLossAttributableToOwnersOfCompany",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Portion of profit or loss attributable to equity holders of the parent company"
        )
    ],
    "ProfitLossAttributableToNoncontrollingInterests": [
        FinancialTag(
            prefix="sg-as",
            element_name="ProfitLossAttributableToNoncontrollingInterests",
            element_id="sg-as_ProfitLossAttributableToNoncontrollingInterests",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Portion of profit or loss attributable to non-controlling interests (minority shareholders)"
        )
    ]
}

# statement-level tags
SG_XBRL_INCOME_STATEMENT_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-as",
        element_name="StatementOfProfitOrLossLineItems",
        element_id="sg-as_StatementOfProfitOrLossLineItems",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container element for the entire statement of profit or loss"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="ProfitLossAbstract",
        element_id="sg-as_ProfitLossAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for the main profit or loss components"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="ProfitLossAttributableToAbstract",
        element_id="sg-as_ProfitLossAttributableToAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for profit or loss attribution breakdown"
    )
]

MANDATORY_INCOME_STATEMENT_TAGS = {
    "Revenue": True,
    "ProfitLossBeforeTaxation": True,
    "TaxExpenseBenefitContinuingOperations": True,
    "ProfitLoss": True,
    "ProfitLossAttributableToOwnersOfCompany": True
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Note - Trade and other receivables XBRL taxonomy tags
# -----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_RECEIVABLES_TAGS = {
    "TradeAndOtherReceivablesAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivablesAbstract",
            element_id="sg-as_TradeAndOtherReceivablesAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element for trade and other receivables note disclosures"
        )
    ],
    "DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesTable": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesTable",
            element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesTable",
            abstract=True,
            data_type="xbrldt:hypercubeItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:hypercubeItem",
            description="Table structure for organizing trade and other receivables disclosures with dimensions"
        )
    ],
    "ConsolidatedAndSeparateFinancialStatementsAxis": [
        FinancialTag(
            prefix="sg-as",
            element_name="ConsolidatedAndSeparateFinancialStatementsAxis",
            element_id="sg-as_ConsolidatedAndSeparateFinancialStatementsAxis",
            abstract=True,
            data_type="xbrldt:dimensionItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:dimensionItem",
            description="Dimension to distinguish between consolidated and separate financial statement data"
        )
    ],
    "DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesLineItems": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesLineItems",
            element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesLineItems",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Container for the line items in the trade and other receivables note"
        )
    ],
    "TradeAndOtherReceivablesDueFromThirdParties": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivablesDueFromThirdParties",
            element_id="sg-as_TradeAndOtherReceivablesDueFromThirdParties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts due from third parties for goods sold or services provided (trade debtors)"
        )
    ],
    "TradeAndOtherReceivablesDueFromRelatedParties": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivablesDueFromRelatedParties",
            element_id="sg-as_TradeAndOtherReceivablesDueFromRelatedParties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts due from related parties such as subsidiaries, associates, or other affiliated entities"
        )
    ],
    "UnbilledReceivables": [
        FinancialTag(
            prefix="sg-as",
            element_name="UnbilledReceivables",
            element_id="sg-as_UnbilledReceivables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts earned but not yet billed to customers (contract assets, accrued revenue)"
        )
    ],
    "OtherReceivables": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherReceivables",
            element_id="sg-as_OtherReceivables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other receivables not specifically classified elsewhere (deposits, loans, advances, etc.)"
        )
    ],
    "TradeAndOtherReceivables": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherReceivables",
            element_id="sg-as_TradeAndOtherReceivables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="debit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total trade and other receivables, representing all receivable amounts combined"
        )
    ]
}

# statement-level tags
SG_XBRL_RECEIVABLES_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-as",
        element_name="TradeAndOtherReceivablesAbstract",
        element_id="sg-as_TradeAndOtherReceivablesAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container for the entire trade and other receivables note"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesLineItems",
        element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherReceivablesLineItems",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for the trade and other receivables breakdown line items"
    )
]

MANDATORY_RECEIVABLES_TAGS = {
    "TradeAndOtherReceivables": True  # Only the total is typically mandatory
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Note - Trade and other payables XBRL taxonomy tags
# -----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_PAYABLES_TAGS = {
    "TradeAndOtherPayablesAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayablesAbstract",
            element_id="sg-as_TradeAndOtherPayablesAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element for trade and other payables note disclosures"
        )
    ],
    "DisclosureOfDetailedInformationAboutTradeAndOtherPayablesTable": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfDetailedInformationAboutTradeAndOtherPayablesTable",
            element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherPayablesTable",
            abstract=True,
            data_type="xbrldt:hypercubeItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:hypercubeItem",
            description="Table structure for organizing trade and other payables disclosures with dimensions"
        )
    ],
    "ConsolidatedAndSeparateFinancialStatementsAxis": [
        FinancialTag(
            prefix="sg-as",
            element_name="ConsolidatedAndSeparateFinancialStatementsAxis",
            element_id="sg-as_ConsolidatedAndSeparateFinancialStatementsAxis",
            abstract=True,
            data_type="xbrldt:dimensionItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:dimensionItem",
            description="Dimension to distinguish between consolidated and separate financial statement data"
        )
    ],
    "DisclosureOfDetailedInformationAboutTradeAndOtherPayablesLineItems": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfDetailedInformationAboutTradeAndOtherPayablesLineItems",
            element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherPayablesLineItems",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Container for the line items in the trade and other payables note"
        )
    ],
    "TradeAndOtherPayablesDueToThirdParties": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayablesDueToThirdParties",
            element_id="sg-as_TradeAndOtherPayablesDueToThirdParties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts owed to third-party suppliers for goods or services received (trade creditors)"
        )
    ],
    "TradeAndOtherPayablesDueToRelatedParties": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayablesDueToRelatedParties",
            element_id="sg-as_TradeAndOtherPayablesDueToRelatedParties",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Amounts owed to related parties such as subsidiaries, associates, or other affiliated entities"
        )
    ],
    "DeferredIncome": [
        FinancialTag(
            prefix="sg-as",
            element_name="DeferredIncome",
            element_id="sg-as_DeferredIncome",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Revenue received but not yet earned (unearned revenue, contract liabilities, advances from customers)"
        )
    ],
    "OtherPayables": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherPayables",
            element_id="sg-as_OtherPayables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Other payables not specifically classified elsewhere (accruals, deferred expenses, etc.)"
        )
    ],
    "TradeAndOtherPayables": [
        FinancialTag(
            prefix="sg-as",
            element_name="TradeAndOtherPayables",
            element_id="sg-as_TradeAndOtherPayables",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="instant",
            substitution_group="xbrli:item",
            description="Total trade and other payables, representing all payable amounts combined"
        )
    ]
}

# statement-level tags
SG_XBRL_PAYABLES_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-as",
        element_name="TradeAndOtherPayablesAbstract",
        element_id="sg-as_TradeAndOtherPayablesAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container for the entire trade and other payables note"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="DisclosureOfDetailedInformationAboutTradeAndOtherPayablesLineItems",
        element_id="sg-as_DisclosureOfDetailedInformationAboutTradeAndOtherPayablesLineItems",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for the trade and other payables breakdown line items"
    )
]

MANDATORY_PAYABLES_TAGS = {
    "TradeAndOtherPayables": True  # Only the total is typically mandatory
}

#----------------------------------------------------------------------------------------------------------------------------------------------
# Note - Revenue XBRL taxonomy tags
# -----------------------------------------------------------------------------------------------------------------------------------------------

SG_XBRL_REVENUE_TAGS = {
    "DisclosureOfRevenueAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfRevenueAbstract",
            element_id="sg-as_DisclosureOfRevenueAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container element for revenue note disclosures"
        )
    ],
    "DisclosureOfRevenueTable": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfRevenueTable",
            element_id="sg-as_DisclosureOfRevenueTable",
            abstract=True,
            data_type="xbrldt:hypercubeItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:hypercubeItem",
            description="Table structure for organizing revenue disclosures with dimensions"
        )
    ],
    "ConsolidatedAndSeparateFinancialStatementsAxis": [
        FinancialTag(
            prefix="sg-as",
            element_name="ConsolidatedAndSeparateFinancialStatementsAxis",
            element_id="sg-as_ConsolidatedAndSeparateFinancialStatementsAxis",
            abstract=True,
            data_type="xbrldt:dimensionItem",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrldt:dimensionItem",
            description="Dimension to distinguish between consolidated and separate financial statement data"
        )
    ],
    "DisclosureOfRevenueLineItems": [
        FinancialTag(
            prefix="sg-as",
            element_name="DisclosureOfRevenueLineItems",
            element_id="sg-as_DisclosureOfRevenueLineItems",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Container for the line items in the revenue note"
        )
    ],
    "RevenueAbstract": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueAbstract",
            element_id="sg-as_RevenueAbstract",
            abstract=True,
            data_type="xbrli:stringItemType",
            balance_type=None,
            period_type="duration",
            substitution_group="xbrli:item",
            description="Abstract container for grouping revenue breakdown items"
        )
    ],
    "RevenueFromPropertyTransferredAtPointInTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromPropertyTransferredAtPointInTime",
            element_id="sg-as_RevenueFromPropertyTransferredAtPointInTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from property sales when control transfers at a specific point in time"
        )
    ],
    "RevenueFromGoodsTransferredAtPointInTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromGoodsTransferredAtPointInTime",
            element_id="sg-as_RevenueFromGoodsTransferredAtPointInTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from sale of goods when control transfers at a specific point in time"
        )
    ],
    "RevenueFromServicesTransferredAtPointInTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromServicesTransferredAtPointInTime",
            element_id="sg-as_RevenueFromServicesTransferredAtPointInTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from services delivered at a specific point in time rather than over time"
        )
    ],
    "RevenueFromPropertyTransferredOverTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromPropertyTransferredOverTime",
            element_id="sg-as_RevenueFromPropertyTransferredOverTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from property development where control transfers gradually over time"
        )
    ],
    "RevenueFromConstructionContractsOverTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromConstructionContractsOverTime",
            element_id="sg-as_RevenueFromConstructionContractsOverTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from construction contracts over time using progress measurements"
        )
    ],
    "RevenueFromServicesTransferredOverTime": [
        FinancialTag(
            prefix="sg-as",
            element_name="RevenueFromServicesTransferredOverTime",
            element_id="sg-as_RevenueFromServicesTransferredOverTime",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue recognized from services that are delivered over time (e.g., subscription services)"
        )
    ],
    "OtherRevenue": [
        FinancialTag(
            prefix="sg-as",
            element_name="OtherRevenue",
            element_id="sg-as_OtherRevenue",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Revenue from other sources not specifically classified elsewhere"
        )
    ],
    "Revenue": [
        FinancialTag(
            prefix="sg-as",
            element_name="Revenue",
            element_id="sg-as_Revenue",
            abstract=False,
            data_type="xbrli:monetaryItemType",
            balance_type="credit",
            period_type="duration",
            substitution_group="xbrli:item",
            description="Total revenue from all sources combined"
        )
    ]
}

# statement-level tags
SG_XBRL_REVENUE_STATEMENT_TAGS = [
    FinancialTag(
        prefix="sg-as",
        element_name="DisclosureOfRevenueAbstract",
        element_id="sg-as_DisclosureOfRevenueAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Parent container for the entire revenue note"
    ),
    FinancialTag(
        prefix="sg-as",
        element_name="RevenueAbstract",
        element_id="sg-as_RevenueAbstract",
        abstract=True,
        data_type="xbrli:stringItemType",
        balance_type=None,
        period_type="duration",
        substitution_group="xbrli:item",
        description="Section container for the revenue breakdown items"
    )
]

MANDATORY_REVENUE_TAGS = {
    "Revenue": True  # Only the total is typically mandatory
}

#### DEPENDENCY INSTANCE

In [ ]:
# Combine

# Combine all tag dictionaries
SG_XBRL_TAXONOMY = {
    **SG_XBRL_FILING_TAGS,
    **SG_XBRL_DIRECTORS_TAGS,
    **SG_XBRL_AUDIT_TAGS,
    **SG_XBRL_FINANCIAL_POSITION_TAGS,  # This already includes liabilities and equity
    **SG_XBRL_INCOME_STATEMENT_TAGS,
    **SG_XBRL_RECEIVABLES_TAGS,
    **SG_XBRL_PAYABLES_TAGS,
    **SG_XBRL_REVENUE_TAGS
}

# Combine all statement-level tags
SG_XBRL_STATEMENT_TAGS = (
    SG_XBRL_FILING_STATEMENT_TAGS + 
    SG_XBRL_DIRECTORS_STATEMENT_TAGS + 
    SG_XBRL_AUDIT_STATEMENT_TAGS + 
    SG_XBRL_FINANCIAL_POSITION_STATEMENT_TAGS +  # This already includes liabilities and equity
    SG_XBRL_INCOME_STATEMENT_STATEMENT_TAGS + 
    SG_XBRL_RECEIVABLES_STATEMENT_TAGS + 
    SG_XBRL_PAYABLES_STATEMENT_TAGS + 
    SG_XBRL_REVENUE_STATEMENT_TAGS
)

# Combine all mandatory field dictionaries
MANDATORY_TAGS = {
    **MANDATORY_FILING_TAGS,
    **MANDATORY_DIRECTORS_TAGS,
    **MANDATORY_AUDIT_TAGS,
    **MANDATORY_FINANCIAL_POSITION_TAGS,  # This already includes liabilities and equity
    **MANDATORY_INCOME_STATEMENT_TAGS,
    **MANDATORY_RECEIVABLES_TAGS,
    **MANDATORY_PAYABLES_TAGS,
    **MANDATORY_REVENUE_TAGS
}

# Create the dependency instance with the comprehensive tag collections
sg_xbrl_deps = XBRLTaxonomyDependencies(
    taxonomy_name="sg-as-2022-02",
    entity_name="Default Company",
    mandatory_fields=MANDATORY_TAGS,
    field_tags=SG_XBRL_TAXONOMY,
    statement_tags=SG_XBRL_STATEMENT_TAGS,
    reporting_year="2022"
)

### SYSTEM PROMPTS

In [ ]:
# Tagging System Prompt
XBRL_DATA_TAGGING_PROMPT = """You are an XBRL data tagging specialist for Singapore financial statements. 
Your task is to apply appropriate XBRL tags to financial data based on pre-mapped concepts from the Singapore ACRA taxonomy (version 2022.2).

## YOUR ROLE

You receive data that has already been mapped to XBRL concepts by another agent. Your job is to:
1. Apply the correct tags to specific data points
2. Add appropriate contextual information to each tagged value
3. Ensure tagging follows Singapore XBRL formatting requirements
4. Validate that tagged data maintains proper calculation relationships

## AVAILABLE TOOLS AND DEPENDENCIES

1. `tag_financial_data`: This tool applies XBRL tags to specific financial data points.
   - Input: Mapped financial item, value, period, and additional context
   - Output: TaggedValue object with the appropriate XBRL tags applied

2. `validate_tagged_data`: This tool verifies that tagged data meets XBRL requirements.
   - Input: Collection of TaggedValue objects
   - Output: Validation results and any issues detected

3. `XBRLTaxonomyDependencies`: Contains comprehensive tag metadata for Singapore XBRL
   - Each tag contains detailed description, data type, balance type, and period type information
   - Used to validate proper tag application

## TAGGING PROCESS

1. PREPARE CONTEXTUAL INFORMATION:
   - Identify reporting periods (current and comparative)
   - Determine whether data is company-level or consolidated
   - Note currency and precision requirements

2. APPLY TAGS to each data point:
   - Use predefined mappings to select the correct tag
   - Apply proper context references (period, entity, dimensions)
   - Format data according to the tag's data type requirements
   - Include units for monetary and numeric values

3. VALIDATE tagged data:
   - Ensure all mandatory tags are included
   - Verify calculation relationships are maintained
   - Check that context references are consistent across related items
   - Confirm data types match tag requirements

4. DOCUMENT any special handling or exceptions:
   - Note any unusual tagging decisions
   - Explain any extensions or custom tags needed
   - Highlight potential validation issues

## SPECIAL CONSIDERATIONS

1. PERIOD TYPES: Apply correct period contexts
   - "instant" tags require a single date context (typically period end date)
   - "duration" tags require a start and end date context

2. DATA TYPES: Format data according to tag requirements
   - monetaryItemType: Currency values with appropriate precision
   - stringItemType: Text values
   - dateItemType: ISO formatted dates (YYYY-MM-DD)
   - booleanItemType: "true" or "false" values

3. SIGNS AND CALCULATIONS: Respect balance types
   - Credit balance items should be positive when they increase (liabilities, equity, revenue)
   - Debit balance items should be positive when they increase (assets, expenses)
   - Totals must match the sum of their components

Focus on accurate and consistent application of tags to ensure the XBRL output is valid and compliant with Singapore ACRA requirements.
"""

### AGENTS

In [ ]:
# SETUP MAPPING AGENT

tagging_model = OpenAIModel(model_name="gpt-4o", api_key=OPENAI_API_KEY)

# Define the agent with dependencies
xbrl_tagging_agent = Agent(
    model=tagging_model,
    result_type=PartialXBRLWithTags,
    system_prompt=XBRL_DATA_TAGGING_PROMPT,
    deps_type=XBRLTaxonomyDependencies,
    retries=10,
)

### TOOLS

#### APPLY TAGS TOOL

In [ ]:
# Tagging apply_tags_to_element tool
@xbrl_tagging_agent.tool
def apply_tags_to_element(
    context: RunContext[XBRLTaxonomyDependencies],
    element_name: str,
    value: Any,
    statement_type: str,
    is_instant: bool = True
) -> Dict[str, Any]:
    """
    Apply appropriate XBRL tags to a financial element
    
    Args:
        context: Runtime context containing taxonomy dependencies
        element_name: The name of the financial element to tag
        value: The value of the element
        statement_type: Type of statement (filing, balance_sheet, income_statement, etc.)
        is_instant: Whether the element is an instant (point-in-time) value
        
    Returns:
        Dictionary containing the tagged value and metadata
    """
    tags = []
    messages = []
    
    # Find tags for the element name
    if element_name in context.deps.field_tags:
        tags = context.deps.field_tags[element_name]
        messages.append(f"Found exact tag match for {element_name}")
    else:
        messages.append(f"No exact tag match found for {element_name}")
        # Try finding similar tags for fuzzy matching
        for field_name, field_tags in context.deps.field_tags.items():
            # Simple substring matching - could be improved with better algorithms
            if element_name.lower() in field_name.lower() or field_name.lower() in element_name.lower():
                tags = field_tags
                messages.append(f"Using similar tag: {field_name}")
                break
    
    # Check if it's a mandatory field
    is_mandatory = False
    if element_name in context.deps.mandatory_fields:
        is_mandatory = context.deps.mandatory_fields[element_name]
        if is_mandatory:
            messages.append(f"Note: {element_name} is a mandatory field")
    
    # Create response
    result = {
        "element_name": element_name,
        "value": value,
        "tags": [tag.dict() for tag in tags],  # Convert tags to dict for JSON serialization
        "is_mandatory": is_mandatory,
        "messages": messages,
    }
    
    return result

#### CREATE CONTEXT TOOL

In [ ]:
# Tagging create_context_info tool
@xbrl_tagging_agent.tool
def create_context_info(
    context: RunContext[XBRLTaxonomyDependencies],  # Add this as first parameter
    entity_name: str,
    entity_identifier: str,
    period_end: date,
    period_start: Optional[date] = None,
    is_consolidated: bool = False,
    dimensions: Dict[str, str] = None
) -> Dict[str, Any]:
    """
    Create XBRL context information for a set of related elements
    
    Args:
        context: Runtime context containing taxonomy dependencies
        entity_name: Name of the reporting entity
        entity_identifier: Unique identifier of the entity (UEN)
        period_end: End date of the reporting period
        period_start: Start date of the reporting period (for duration contexts)
        is_consolidated: Whether the context is for consolidated statements
        dimensions: Additional dimensions for the context
        
    Returns:
        Dictionary containing context information
    """
    # Create a unique context ID based on parameters
    period_part = f"i{period_end.strftime('%Y%m%d')}" if period_start is None else \
                  f"d{period_start.strftime('%Y%m%d')}to{period_end.strftime('%Y%m%d')}"
    
    context_id = f"ctx_{period_part}_{'c' if is_consolidated else 's'}"
    
    # Add dimension information if provided
    if dimensions:
        dim_parts = []
        for dim_name, dim_value in sorted(dimensions.items()):
            dim_parts.append(f"{dim_name}-{dim_value}")
        if dim_parts:
            context_id = f"{context_id}_{'_'.join(dim_parts)}"
    
    # Create context object
    context_info = {
        "id": context_id,
        "entity": {
            "name": entity_name,
            "identifier": entity_identifier
        },
        "period": {
            "end_date": period_end.isoformat()
        },
        "is_consolidated": is_consolidated
    }
    
    if period_start is not None:
        context_info["period"]["start_date"] = period_start.isoformat()
    
    if dimensions:
        context_info["dimensions"] = dimensions
    
    return context_info

#### TAG STATEMENT TOOL

In [ ]:
# Tagging tag_statement_section tool
@xbrl_tagging_agent.tool
def tag_statement_section(
    context: RunContext[XBRLTaxonomyDependencies],
    section_name: str,
    section_data: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Apply tags to an entire statement section
    
    Args:
        context: Runtime context containing taxonomy dependencies
        section_name: Name of the section (e.g., "filingInformation", "statementOfFinancialPosition")
        section_data: Dictionary of data for the section
        
    Returns:
        Dictionary with tagged section data
    """
    tagged_section = {"meta_tags": []}
    
    # Find section-level abstract tags
    for tag in context.deps.statement_tags:
        if section_name.lower() in tag.element_name.lower():
            tagged_section["meta_tags"].append(tag.dict())
    
    # Tag individual elements
    for element_name, element_value in section_data.items():
        # Skip processing of lists/dicts - they would be handled separately
        if isinstance(element_value, (list, dict)):
            continue
            
        # Find matching tags
        tags = []
        if element_name in context.deps.field_tags:
            tags = context.deps.field_tags[element_name]
        
        tagged_section[element_name] = {
            "value": element_value,
            "tags": [tag.dict() for tag in tags]
        }
    
    return tagged_section

#### VALIDATE TAGGED DATA TOOL


In [ ]:
# Tagging validate_tagged_data tool
@xbrl_tagging_agent.tool
def validate_tagged_data(
    context: RunContext[XBRLTaxonomyDependencies],
    tagged_data: Dict[str, Any]
) -> List[Dict[str, str]]:
    """
    Validate the tagged data for completeness and correctness
    
    Args:
        context: Runtime context containing taxonomy dependencies
        tagged_data: Complete set of tagged financial data
        
    Returns:
        List of validation issues found
    """
    issues = []
    
    # Check for missing mandatory fields
    for field_name, is_mandatory in context.deps.mandatory_fields.items():
        if is_mandatory:
            field_found = False
            
            # Search through all sections for the field
            for section_name, section_data in tagged_data.items():
                if isinstance(section_data, dict) and field_name in section_data:
                    field_found = True
                    break
            
            if not field_found:
                issues.append({
                    "type": "missing_mandatory_field",
                    "field": field_name,
                    "message": f"Mandatory field '{field_name}' is missing from the tagged data"
                })
    
    # Check for fields without tags
    for section_name, section_data in tagged_data.items():
        if isinstance(section_data, dict):
            for field_name, field_data in section_data.items():
                if isinstance(field_data, dict) and "tags" in field_data and not field_data["tags"]:
                    issues.append({
                        "type": "missing_tags",
                        "section": section_name,
                        "field": field_name,
                        "message": f"No tags applied to field '{field_name}' in section '{section_name}'"
                    })
    
    return issues

### USAGE EXAMPLE

In [ ]:
# Parse the mapped data if it's in JSON format
try:
    if isinstance(result_mapping.data, str):
        mapped_data = json.loads(result_mapping.data)
    elif hasattr(result_mapping, 'data') and isinstance(result_mapping.data, dict):
        mapped_data = result_mapping.data
    elif isinstance(result_mapping, dict):
        mapped_data = result_mapping
    else:
        # Fall back to the raw result_mapping if none of the above
        mapped_data = result_mapping
    
    # Print the structure to help debug
    print(f"Type of mapped_data: {type(mapped_data)}")
    if isinstance(mapped_data, dict):
        print(f"Keys in mapped_data: {list(mapped_data.keys())}")
    
    # Create parameters for the tagging context with safer access
    tagging_params = {}
    
    # Check if we can access filingInformation
    if isinstance(mapped_data, dict):
        if "filingInformation" in mapped_data:
            filing_info = mapped_data["filingInformation"]
            # Use .get() with defaults to avoid KeyError
            tagging_params = {
                "entity_name": filing_info.get("NameOfCompany", "Unknown Company"),
                "entity_identifier": filing_info.get("UniqueEntityNumber", "Unknown"),
                "current_period_end": date.fromisoformat(filing_info.get("CurrentPeriodEndDate", "2022-12-31")),
                "current_period_start": date.fromisoformat(filing_info.get("CurrentPeriodStartDate", "2022-01-01")),
                "is_consolidated": filing_info.get("NatureOfFinancialStatementsCompanyLevelOrConsolidated", "") == "Consolidated"
            }
        # Alternative structures we might encounter
        elif hasattr(mapped_data, "filingInformation"):
            # Handle case where it's an object with attributes
            filing_info = mapped_data.filingInformation
            tagging_params = {
                "entity_name": getattr(filing_info, "NameOfCompany", "Unknown Company"),
                "entity_identifier": getattr(filing_info, "UniqueEntityNumber", "Unknown"),
                # And so on for other fields...
            }
        else:
            print("Warning: Could not find filingInformation in the data structure")
            print(f"Available top-level keys/attributes: {dir(mapped_data) if not isinstance(mapped_data, dict) else list(mapped_data.keys())}")
    else:
        print(f"Warning: mapped_data is not a dictionary but a {type(mapped_data)}")
        if hasattr(mapped_data, "filingInformation"):
            print("However, it does have a filingInformation attribute")
        
except Exception as e:
    print(f"Error processing mapped data: {e}")
    import traceback
    traceback.print_exc()
    tagging_params = {
        "entity_name": "Unknown Company",
        "entity_identifier": "Unknown",
        "current_period_end": date.today(),
        "current_period_start": date(date.today().year, 1, 1),
        "is_consolidated": False
    }

In [ ]:
# Sample test run - Minimal version - only pass dependencies
tagged_result = await xbrl_tagging_agent.run(
    "Apply appropriate XBRL tags to the following financial data: " + str(mapped_data)[:4000],  # Include data in prompt
    deps=sg_xbrl_deps  # Just dependencies
)

In [ ]:
# Print Test Run
debug(tagged_result)
print("\nTagged Statement:")
pprint.pprint(tagged_result.data)